In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import plotly.express as px
import matplotlib.pyplot as plt
import random
import missingno as msno
import statsmodels.api as sm
from collections import Counter
from sklearn import preprocessing
from random import sample
from sklearn.pipeline import Pipeline
from sklearn import metrics, linear_model
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, auc, accuracy_score, roc_auc_score, mean_squared_error, mean_absolute_error, log_loss
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Slicing data
from sklearn.model_selection import StratifiedShuffleSplit

# Cross-Validation techniques
from sklearn.model_selection import KFold, StratifiedKFold

# Principal Analysis
from sklearn.decomposition import PCA
from numpy.linalg import eigh
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Deal with imbalance data
from sklearn.utils import resample 
from sklearn.datasets import make_classification

# Resampling method
# RUS
from imblearn.under_sampling import RandomUnderSampler
# ROS
from imblearn.over_sampling import RandomOverSampler
# SMOTE
from imblearn.over_sampling import SMOTE

# Common machine learning methods
# Linear regression
from sklearn.linear_model import LinearRegression
# Logistic Regression
from sklearn.linear_model import LogisticRegression 
from matplotlib import pyplot
# KNN
from sklearn.neighbors import KNeighborsClassifier
# SVM
from sklearn import svm
from sklearn.svm import SVC
# Decision tree
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor 
from sklearn.tree import export_graphviz
from IPython.display import Image 
# Neural Networks 
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.utils import get_custom_objects

# Bagging
from sklearn.ensemble import BaggingClassifier
# Random Forest
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
# Bagged NN
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier

# Boosting
# Light GBM
from lightgbm import LGBMClassifier
# Extreme Gradient Boosting
import xgboost as xgb
from xgboost import XGBClassifier
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# Adaptive Boosting
from sklearn.ensemble import AdaBoostClassifie


# Pre-processing data function

In [ ]:
# Descriptive Statistics
def data_description(df):
    print("Infomation of the dataset:")
    print(df.info())
    print()
    
    print("The number of null value in each column:")
    print(df.isnull().sum())
    print()
    
    print("The percentage of null value in each column:")
    print((df.isnull().sum()/len(df)).sort_values(ascending = False))  

In [ ]:
# map with previous_application, credit_card_balance, installments_payments
def POS_CASH_balance_preprocessing(df):
    data_description(df)
    print()
    index_names = df[(df['NAME_CONTRACT_STATUS'] == 'Canceled') | (df['NAME_CONTRACT_STATUS'] == 'XNA')].index
    df = df.drop(index_names)
    print("NAME_CONTRACT_STATUS after removing Canceled and XNA")
    print(df['NAME_CONTRACT_STATUS'].value_counts())
    # Get dummies value for name of contract status
    dummies_NAME_CONTRACT_STATUS = pd.get_dummies(df['NAME_CONTRACT_STATUS'])
    dummies_NAME_CONTRACT_STATUS.columns = dummies_NAME_CONTRACT_STATUS.columns.str.replace(' ', '_').str.upper()
    df = pd.concat([df, dummies_NAME_CONTRACT_STATUS], axis = 1)
    # Computation metrics
    df['DPD30PLUS'] = df['SK_DPD'].apply(lambda x: 1 if x >= 30 else 0)
    df['DPD60PLUS'] = df['SK_DPD'].apply(lambda x: 1 if x >= 60 else 0)
    df['DPD90PLUS'] = df['SK_DPD'].apply(lambda x: 1 if x >= 90 else 0)
    df['DPD120PLUS'] = df['SK_DPD'].apply(lambda x: 1 if x >= 120 else 0)
    
    df['DPD30PLUS_DEF'] = df['SK_DPD_DEF'].apply(lambda x: 1 if x >= 30 else 0)
    df['DPD60PLUS_DEF'] = df['SK_DPD_DEF'].apply(lambda x: 1 if x >= 60 else 0)
    df['DPD90PLUS_DEF'] = df['SK_DPD_DEF'].apply(lambda x: 1 if x >= 90 else 0)
    df['DPD120PLUS_DEF'] = df['SK_DPD_DEF'].apply(lambda x: 1 if x >= 120 else 0)
    
    epsilon = 0
    df['PER_INSTALLMENT_PAID'] = (df['CNT_INSTALMENT'] - df['CNT_INSTALMENT_FUTURE'])/(df['CNT_INSTALMENT']+epsilon)
    df['PER_INSTALLMENT_FUTURE'] = df['CNT_INSTALMENT_FUTURE']/(df['CNT_INSTALMENT']+epsilon)
    
    POS_CASH_balance_agg_dict = {
        'SK_ID_PREV': ['count'],
        'MONTHS_BALANCE': ['max', 'min', 'mean'],
        'CNT_INSTALMENT': ['max', 'min', 'mean'],
        'CNT_INSTALMENT_FUTURE': ['max', 'min', 'mean'],
        'SK_DPD': ['max', 'min', 'mean'],
        'SK_DPD_DEF': ['max', 'min', 'mean'],
    }
    
    POS_CASH_balance_group = df.groupby('SK_ID_CURR') 
    POS_CASH_balance_agg = POS_CASH_balance_group.agg(POS_CASH_balance_agg_dict)
    POS_CASH_balance_agg.columns = ['PCB_'+('_').join(column).upper() for column in POS_CASH_balance_agg.columns.ravel()]
    POS_CASH_balance_agg = POS_CASH_balance_agg.reset_index()
    print('POS_CASH_balance_agg:', POS_CASH_balance_agg.shape)
    
    return POS_CASH_balance_agg

In [ ]:
def installments_payments_preprocessing(df):
    data_description(df)
    print()
    df = df.fillna(0)
    df['SK_DPD'] = df['DAYS_ENTRY_PAYMENT'] - df['DAYS_INSTALMENT']
    df['UNPAID_AMT'] = df['AMT_INSTALMENT'] - df['AMT_PAYMENT']
    df['PER_OVD_AMT'] = (df['AMT_INSTALMENT'] - df['AMT_PAYMENT'])/df['AMT_INSTALMENT']
    # Computation metrics
    df['DPD30PLUS'] = df['SK_DPD'].apply(lambda x: 1 if x >= 30 else 0)
    df['DPD90PLUS'] = df['SK_DPD'].apply(lambda x: 1 if x >= 90 else 0)
    df['DPD120PLUS'] = df['SK_DPD'].apply(lambda x: 1 if x >= 120 else 0)
    # Agg dict
    agg_dict = {
        'SK_ID_PREV': ['count'],
        'NUM_INSTALMENT_VERSION':['nunique'],
        'DAYS_ENTRY_PAYMENT':['mean', 'max', 'sum'],
        'DAYS_INSTALMENT':['mean', 'max', 'sum'],
        'AMT_INSTALMENT':['mean', 'max', 'sum'],
        'AMT_PAYMENT':['mean', 'max','sum'],
        'SK_DPD':['mean', 'max', 'sum'],
        'UNPAID_AMT':['mean', 'max', 'sum'],
        'PER_OVD_AMT':['mean', 'max','sum'],
        'DPD30PLUS': ['mean', 'max'],
        'DPD90PLUS': ['mean', 'max'],
        'DPD120PLUS': ['mean', 'max'],
    }
    print(df.info())
    installments_payments_group = df.groupby(['SK_ID_CURR']) 
    installments_payments_agg = installments_payments_group.agg(agg_dict)
    installments_payments_agg.columns = ['INS_PAYMENT'+('_').join(column).upper() for column in installments_payments_agg.columns.ravel()]
    installments_payments_agg = installments_payments_agg.reset_index()
    print('installments_payments_agg shape:', installments_payments_agg.shape)
    
    return installments_payments_agg

In [ ]:
def previous_application_preprocessing(df):
    data_description(df)
    print()
    # Replace XNA with nan
    df = df.replace('XNA', np.NaN)
    
    # Mapping value for FLAG_LAST_APPL_PER_CONTRACT
    d = {'Y': 0, 'N': 1}
    df['FLAG_LAST_APPL_PER_CONTRACT'] = df['FLAG_LAST_APPL_PER_CONTRACT'].map(d)
    
    # Get dummies value for name of contract status
    get_dummies_column = ['NAME_CONTRACT_TYPE', 'NAME_CONTRACT_STATUS', 'NAME_CLIENT_TYPE', 'NAME_PORTFOLIO', 'NAME_YIELD_GROUP']
    for i in get_dummies_column:
        df_dummies = pd.DataFrame()
        df_dummies = pd.get_dummies(df[i])
        df_dummies.columns = df_dummies.columns.str.replace(' ', '_').str.upper()
        df_dummies = df_dummies.add_prefix(f'{i}_')
        df = pd.concat([df, df_dummies], axis = 1)
    
    # Replace columns value 
    replace_value_columns = ['DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']
    for i in replace_value_columns:
        df[i].replace(365243, np.NaN, inplace = True)
    
    # Computation metrics
    df['PER_SELF_EVALUATION_AMT'] = df['AMT_ANNUITY']/df['AMT_APPLICATION']
    df['PER_FI_EVALUATION_AMT'] = df['AMT_ANNUITY']/df['AMT_CREDIT']
    df['PER_DOWN_PAYMENT'] = df['AMT_DOWN_PAYMENT']/df['AMT_GOODS_PRICE']
    df['PER_GOODS_PRICE_CREDIT'] = df['AMT_GOODS_PRICE']/df['AMT_CREDIT']
    df['PER_APPLICATION_CREDIT'] = df['AMT_APPLICATION']/df['AMT_CREDIT']
    df['DAY_DIFF_LAST_FIRST_DUE'] = df['DAYS_LAST_DUE'] - df['DAYS_FIRST_DUE']
    df['FULL_PAY_AMOUNT'] = df['AMT_ANNUITY'] * df['CNT_PAYMENT']
    
    agg_dict = {
        'SK_ID_PREV' : ['count'],
        'AMT_ANNUITY' : ['max', 'min', 'mean'],
        'AMT_APPLICATION' : ['max', 'min', 'mean'],
        'AMT_CREDIT' : ['max', 'min', 'mean'],
        'AMT_DOWN_PAYMENT' : ['max', 'min', 'mean'],
        'AMT_GOODS_PRICE' : ['max', 'min', 'mean'],
        'NFLAG_LAST_APPL_IN_DAY': ['mean'],
        'RATE_DOWN_PAYMENT': ['max', 'min', 'mean'],
        'SELLERPLACE_AREA' : ['max', 'min', 'mean'],
        'CNT_PAYMENT' : ['max', 'min', 'mean'],
        'DAYS_FIRST_DRAWING' : ['max', 'min', 'mean'],
        'DAYS_FIRST_DUE' : ['max', 'min', 'mean'],
        'DAYS_LAST_DUE_1ST_VERSION' : ['max', 'min', 'mean'],
        'DAYS_LAST_DUE' : ['max', 'min', 'mean'],
        'DAYS_TERMINATION' : ['max', 'min', 'mean'],
        'NFLAG_INSURED_ON_APPROVAL' : ['max', 'mean'],
        'NAME_CONTRACT_TYPE_CASH_LOANS' : ['mean'],
        'NAME_CONTRACT_TYPE_CONSUMER_LOANS' : ['mean'],
        'NAME_CONTRACT_TYPE_REVOLVING_LOANS' : ['mean'],
        'NAME_CONTRACT_STATUS_APPROVED' : ['mean'],
        'NAME_CONTRACT_STATUS_CANCELED' : ['mean'],
        'NAME_CONTRACT_STATUS_REFUSED' : ['mean'],
        'NAME_CONTRACT_STATUS_UNUSED_OFFER' : ['mean'],
        'NAME_CLIENT_TYPE_NEW' : ['mean'],
        'NAME_CLIENT_TYPE_REFRESHED' : ['mean'],
        'NAME_CLIENT_TYPE_REPEATER' : ['mean'],
        'NAME_PORTFOLIO_CARDS' : ['mean'],
        'NAME_PORTFOLIO_CARS' : ['mean'],
        'NAME_PORTFOLIO_CASH': ['mean'],
        'NAME_PORTFOLIO_POS' : ['mean'],
        'NAME_YIELD_GROUP_HIGH' : ['mean'],
        'NAME_YIELD_GROUP_LOW_ACTION' : ['mean'],
        'NAME_YIELD_GROUP_LOW_NORMAL' : ['mean'],        
        'NAME_YIELD_GROUP_MIDDLE' : ['mean'],
        'PER_SELF_EVALUATION_AMT' : ['mean'],
        'PER_FI_EVALUATION_AMT' : ['mean'],
        'PER_DOWN_PAYMENT' : ['mean'],
        'PER_GOODS_PRICE_CREDIT' : ['mean'],
        'PER_APPLICATION_CREDIT' : ['mean'],
        'DAY_DIFF_LAST_FIRST_DUE' : ['max', 'min', 'mean'],
        'FULL_PAY_AMOUNT' : ['max', 'min', 'mean']
    }
    
    previous_application_group = df.groupby('SK_ID_CURR') 
    previous_application_agg = previous_application_group.agg(agg_dict)
    previous_application_agg.columns = ['PREV_APP_'+('_').join(column).upper() for column in previous_application_agg.columns.ravel()]
    previous_application_agg = previous_application_agg.reset_index()
    print('previous_application_agg shape:', previous_application_agg.shape)
    
    return previous_application_agg

In [ ]:
def credit_card_balance_preprocessing(df):
    data_description(df)
    print()
    # select numeric columns
    numeric_columns = df.select_dtypes(include=['number']).columns
    # fill 0 to all NaN 
    df[numeric_columns] = df[numeric_columns].fillna(0)
    
    # Get dummies value for NAME_OF_CONTRACT_STATUS
    df_dummies = pd.DataFrame()
    df_dummies = pd.get_dummies(df['NAME_CONTRACT_STATUS'])
    df_dummies.columns = df_dummies.columns.str.replace(' ', '_').str.upper()
    df_dummies = df_dummies.add_prefix('CONTRACT_STATUS_')
    df = pd.concat([df, df_dummies], axis = 1)
    df = df.drop(['NAME_CONTRACT_STATUS'], axis = 1)
    
    # DPD Flag
    df['DPD30PLUS'] = df['SK_DPD'].apply(lambda x: 1 if x >= 30 else 0)
    df['DPD60PLUS'] = df['SK_DPD'].apply(lambda x: 1 if x >= 60 else 0)
    df['DPD90PLUS'] = df['SK_DPD'].apply(lambda x: 1 if x >= 90 else 0)
    df['DPD120PLUS'] = df['SK_DPD'].apply(lambda x: 1 if x >= 120 else 0)
    
    df['DPD30PLUS_DEF'] = df['SK_DPD_DEF'].apply(lambda x: 1 if x >= 30 else 0)
    df['DPD60PLUS_DEF'] = df['SK_DPD_DEF'].apply(lambda x: 1 if x >= 60 else 0)
    df['DPD90PLUS_DEF'] = df['SK_DPD_DEF'].apply(lambda x: 1 if x >= 90 else 0)
    df['DPD120PLUS_DEF'] = df['SK_DPD_DEF'].apply(lambda x: 1 if x >= 120 else 0)
    
    # Computation metrics
    df['AMT_OUTSTANDING'] = df['AMT_CREDIT_LIMIT_ACTUAL'] - df['AMT_BALANCE']
    df['PER_UTILIZATION'] = df['AMT_OUTSTANDING']/df['AMT_CREDIT_LIMIT_ACTUAL']
    df['PER_AMT_DRAWINGS_ATM_CURRENT_OVER_AMT_OUTSTANDING'] = df['AMT_DRAWINGS_ATM_CURRENT'] / df['AMT_OUTSTANDING']
    df['PER_AMT_DRAWINGS_ATM_CURRENT_OVER_CREDIT_LIMIT'] = df['AMT_DRAWINGS_ATM_CURRENT'] / df['AMT_CREDIT_LIMIT_ACTUAL']
    df['PER_AMT_DRAWINGS_CURRENT_OVER_AMT_OUTSTANDING'] = df['AMT_DRAWINGS_CURRENT'] / df['AMT_OUTSTANDING']
    df['PER_AMT_DRAWINGS_CURRENT_OVER_CREDIT_LIMIT'] = df['AMT_DRAWINGS_CURRENT'] / df['AMT_CREDIT_LIMIT_ACTUAL']
    df['PER_AMT_DRAWINGS_POS_CURRENT'] = df['AMT_DRAWINGS_POS_CURRENT'] / df['AMT_DRAWINGS_CURRENT']
    df['PER_AMT_DRAWINGS_ATM_CURRENT'] = df['AMT_DRAWINGS_ATM_CURRENT'] / df['AMT_DRAWINGS_CURRENT']    
    df['PER_AMT_DRAWINGS_OTHER_CURRENT'] = df['AMT_DRAWINGS_OTHER_CURRENT'] / df['AMT_DRAWINGS_CURRENT']
    df['PER_INS_MIN'] = df['AMT_INST_MIN_REGULARITY'] / df['AMT_OUTSTANDING']
    df['PER_AMT_PAYMENT_CURRENT'] = df['AMT_PAYMENT_CURRENT'] / df['AMT_OUTSTANDING']
    df['PER_AMT_PAYMENT_TOTAL_CURRENT'] = df['AMT_PAYMENT_TOTAL_CURRENT'] / df['AMT_OUTSTANDING']
    df['PER_RECEIVABLE_TOTAL'] = df['AMT_TOTAL_RECEIVABLE'] / df['AMT_OUTSTANDING']
    df['PER_RECEIVABLE_PRINCIPAL'] = df['AMT_RECEIVABLE_PRINCIPAL'] / df['AMT_OUTSTANDING']
    df['PER_CNT_DRAWINGS_ATM_CURRENT'] = df['CNT_DRAWINGS_ATM_CURRENT'] / df['CNT_DRAWINGS_CURRENT']
    df['PER_CNT_DRAWINGS_OTHER_CURRENT'] = df['CNT_DRAWINGS_OTHER_CURRENT'] / df['CNT_DRAWINGS_CURRENT']
    df['PER_CNT_DRAWINGS_POS_CURRENT'] = df['CNT_DRAWINGS_POS_CURRENT'] / df['CNT_DRAWINGS_CURRENT']
    
    agg_dict = {
        'SK_ID_PREV': ['count'],
        'MONTHS_BALANCE': ['max', 'min', 'mean'],
        'AMT_CREDIT_LIMIT_ACTUAL': ['max', 'min', 'mean'],
        'CNT_DRAWINGS_ATM_CURRENT': ['max', 'min', 'mean'],
        'CNT_DRAWINGS_CURRENT': ['max', 'min', 'mean'],
        'CNT_DRAWINGS_OTHER_CURRENT': ['max', 'min', 'mean'],
        'CNT_DRAWINGS_POS_CURRENT': ['max', 'min', 'mean'],
        'CNT_INSTALMENT_MATURE_CUM': ['max', 'min', 'mean'],
        'SK_DPD': ['max', 'min', 'mean'],
        'SK_DPD_DEF': ['max', 'min', 'mean'],
        'CONTRACT_STATUS_ACTIVE': ['max', 'mean'],
        'CONTRACT_STATUS_APPROVED': ['max', 'mean'],
        'CONTRACT_STATUS_COMPLETED': ['max', 'mean'],
        'CONTRACT_STATUS_DEMAND': ['max', 'mean'],
        'CONTRACT_STATUS_REFUSED': ['max', 'mean'],
        'CONTRACT_STATUS_SENT_PROPOSAL': ['max', 'mean'],
        'CONTRACT_STATUS_SIGNED': ['max', 'mean'],
        'DPD30PLUS': ['max', 'mean'],
        'DPD90PLUS': ['max', 'mean'],
        'DPD120PLUS': ['max', 'mean'],
        'DPD30PLUS_DEF': ['max', 'mean'],
        'DPD90PLUS_DEF': ['max', 'mean'],
        'DPD120PLUS_DEF': ['max', 'mean'],
        'PER_UTILIZATION': ['max', 'min', 'mean'],
        'PER_AMT_DRAWINGS_ATM_CURRENT_OVER_AMT_OUTSTANDING': ['max', 'min', 'mean'],
        'PER_AMT_DRAWINGS_ATM_CURRENT_OVER_CREDIT_LIMIT': ['max', 'min', 'mean'],
        'PER_AMT_DRAWINGS_CURRENT_OVER_AMT_OUTSTANDING': ['max', 'min', 'mean'],
        'PER_AMT_DRAWINGS_CURRENT_OVER_CREDIT_LIMIT': ['max', 'min', 'mean'],
        'PER_AMT_DRAWINGS_POS_CURRENT': ['max', 'min', 'mean'],
        'PER_AMT_DRAWINGS_ATM_CURRENT': ['max', 'min', 'mean'],
        'PER_AMT_DRAWINGS_OTHER_CURRENT': ['max', 'min', 'mean'],
        'PER_INS_MIN': ['max', 'min', 'mean'],
        'PER_AMT_PAYMENT_CURRENT': ['max', 'min', 'mean'],
        'PER_AMT_PAYMENT_TOTAL_CURRENT': ['max', 'min', 'mean'],
        'PER_RECEIVABLE_TOTAL': ['max', 'min', 'mean'],
        'PER_RECEIVABLE_PRINCIPAL': ['max', 'min', 'mean'],
        'PER_CNT_DRAWINGS_ATM_CURRENT': ['max', 'min', 'mean'],
        'PER_CNT_DRAWINGS_OTHER_CURRENT': ['max', 'min', 'mean'],
        'PER_CNT_DRAWINGS_POS_CURRENT': ['max', 'min', 'mean']
    }
    credit_card_balance_group = df.groupby('SK_ID_CURR') 
    credit_card_balance_agg = credit_card_balance_group.agg(agg_dict)
    credit_card_balance_agg.columns = ['CCB_'+('_').join(column).upper() for column in credit_card_balance_agg.columns.ravel()]
    credit_card_balance_agg = credit_card_balance_agg.reset_index()
    print('credit_card_balance_agg shape:', credit_card_balance_agg.shape)
    
    return credit_card_balance_agg

In [ ]:
def bureau_preprocessing(bureau, bureau_balance):
    print("Data type info of bureau:")
    print(bureau.info())
    arr1 = np.where(bureau['DAYS_CREDIT_ENDDATE'].isnull() & bureau['DAYS_ENDDATE_FACT'].notnull(), 0, bureau['DAYS_CREDIT_ENDDATE'])
    bureau['DAYS_CREDIT_ENDDATE'] = arr1
    arr2 = np.where(bureau['DAYS_CREDIT_ENDDATE'].notnull() & bureau['DAYS_ENDDATE_FACT'].isnull(), 0, bureau['DAYS_ENDDATE_FACT'])
    bureau['DAYS_ENDDATE_FACT'] = arr2
    # bureau_balance = pd.read_csv(bureau_balance_path)
    print("Data type info of bureau_balance:")
    print(bureau_balance.info())
    # Inner join 
    bureau_total = pd.merge(bureau, bureau_balance, how="inner", on=["SK_ID_BUREAU"])

    # CREDIT_CURRENCY
    mapping = {'currency 1':1,
               'currency 2':2,
               'currency 3':3,
               'currency 4':4
              }
    bureau_total['CREDIT_CURRENCY'] = bureau_total['CREDIT_CURRENCY'].map(mapping) 

    # Num metrics
    # Number of days from the time of applying the loan at CB (the loan period may not be closed) until the payment due date
    bureau_total['DAYS_DIFF_CREDIT_ENDDATE'] = bureau_total['DAYS_CREDIT'] - bureau_total['DAYS_CREDIT_ENDDATE']
    # Số ngày kể từ lúc apply khoảng vay ở CB (khoảng vay đã close) đến khi đến hạn thanh toán
    bureau_total['DAYS_DIFF_CREDIT_ENDDATE_FACT'] = bureau_total['DAYS_CREDIT'] - bureau_total['DAYS_ENDDATE_FACT']
    bureau_total['DAYS_DIFF_ENDDATE_FACT_CREDIT_ENDDATE'] = bureau_total['DAYS_ENDDATE_FACT'] - bureau_total['DAYS_CREDIT_ENDDATE']

    bureau_total['STATUS_CLOSED'] = bureau_total['STATUS'].apply(lambda x: 1 if x == 'C' else 0)
    bureau_total['STATUS_UNKNOWN'] = bureau_total['STATUS'].apply(lambda x: 1 if x == 'X' else 0)
    bureau_total['DPD0'] = bureau_total['STATUS'].apply(lambda x: 1 if x == '0' else 0)
    bureau_total['DPD1_TO_30'] = bureau_total['STATUS'].apply(lambda x: 1 if x == '1' else 0)
    bureau_total['DPD31_TO_60'] = bureau_total['STATUS'].apply(lambda x: 1 if x == '2' else 0)
    bureau_total['DPD61_TO_90'] = bureau_total['STATUS'].apply(lambda x: 1 if x == '3' else 0)
    bureau_total['DPD91_TO_120'] = bureau_total['STATUS'].apply(lambda x: 1 if x == '4' else 0)
    bureau_total['DPD120PLUS_SOLD_WRITEOFF'] = bureau_total['STATUS'].apply(lambda x: 1 if x == '5' else 0)

    bureau_total['HAS_DPD'] = bureau_total['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 0 else 0)
    bureau_total['HAS_DPD30PLUS'] = bureau_total['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x >= 30 else 0)
    bureau_total['HAS_DPD60PLUS'] = bureau_total['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x >= 60 else 0)
    bureau_total['HAS_DPD90PLUS'] = bureau_total['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x >= 90 else 0)
    bureau_total['HAS_DPD_OVER120'] = bureau_total['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 120 else 0)

    # Add in metrics
    # Percentage metrics
    epsilon = 0
    # The total owned amt over the total credit limit
    bureau_total['PER_DEBT_AMT_CREDIT'] = bureau_total['AMT_CREDIT_SUM_DEBT']/(bureau_total['AMT_CREDIT_SUM']+epsilon)
    # The total owned amt over the total credit limit of credit card
    bureau_total['PER_DEBT_AMT_CREDIT_CARD'] = bureau_total['AMT_CREDIT_SUM_DEBT']/(bureau_total['AMT_CREDIT_SUM_LIMIT']+epsilon)
    # The total overdue amount over the total debt
    bureau_total['PER_CREDIT_OVD_AMT_DEBT'] = bureau_total['AMT_CREDIT_SUM_OVERDUE']/(bureau_total['AMT_CREDIT_SUM_DEBT']+epsilon)
    # The total overdue amount over the total credit limit of credit card
    bureau_total['PER_CREDIT_OVD_AMT_CREDIT_CARD_LIMIT'] = bureau_total['AMT_CREDIT_SUM_OVERDUE']/(bureau_total['AMT_CREDIT_SUM_LIMIT']+epsilon)
    # The total overdue amount over the total credit limit
    bureau_total['PER_CREDIT_OVD_AMT_CREDIT_LIMIT'] = bureau_total['AMT_CREDIT_SUM_OVERDUE']/(bureau_total['AMT_CREDIT_SUM']+epsilon)
    # The monthly payment amt over the total credit limit
    bureau_total['PER_ANNUITY_TOTAL_CREDIT'] = bureau_total['AMT_ANNUITY']/(bureau_total['AMT_CREDIT_SUM']+epsilon)
    # The monthly payment amt over the total credit limit of credit card
    bureau_total['PER_ANNUITY_CREDIT_CARD'] = bureau_total['AMT_ANNUITY']/(bureau_total['AMT_CREDIT_SUM_LIMIT']+epsilon)
    # The maximum overdue amt over total debt 
    bureau_total['PER_MAX_CREDIT_OVD_AMT_DEBT'] = bureau_total['AMT_CREDIT_MAX_OVERDUE']/(bureau_total['AMT_CREDIT_SUM_DEBT']+epsilon)
    # The maximum overdue amt over the credit limit of credit card
    bureau_total['PER_MAX_CREDIT_OVD_AMT_CREDIT_CARD_LIMIT'] = bureau_total['AMT_CREDIT_MAX_OVERDUE']/(bureau_total['AMT_CREDIT_SUM_LIMIT']+epsilon)
    # The maximum overdue amt over the total credit limit
    bureau_total['PER_MAX_CREDIT_OVD_AMT_CREDIT_LIMIT'] = bureau_total['AMT_CREDIT_MAX_OVERDUE']/(bureau_total['AMT_CREDIT_SUM']+epsilon)

    bureau_agg_dict = {
        'SK_ID_BUREAU':['count'],
        'DAYS_CREDIT':['min', 'max', 'mean'],
        'CREDIT_DAY_OVERDUE':['min', 'max', 'mean'],
        'DAYS_CREDIT_ENDDATE':['min', 'max', 'mean'],
        'DAYS_ENDDATE_FACT':['min', 'max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean', 'sum'],

        'DAYS_DIFF_CREDIT_ENDDATE':['min', 'max', 'mean'],
        'DAYS_DIFF_CREDIT_ENDDATE_FACT':['min', 'max', 'mean'],
        'DAYS_DIFF_ENDDATE_FACT_CREDIT_ENDDATE':['min', 'max', 'mean'],

        'PER_DEBT_AMT_CREDIT':['min', 'max', 'mean'],
        'PER_DEBT_AMT_CREDIT_CARD':['min', 'max', 'mean'],
        'PER_CREDIT_OVD_AMT_DEBT':['min', 'max', 'mean'],
        'PER_CREDIT_OVD_AMT_CREDIT_CARD_LIMIT':['min', 'max', 'mean'],
        'PER_CREDIT_OVD_AMT_CREDIT_LIMIT':['min', 'max', 'mean'],
        'PER_ANNUITY_TOTAL_CREDIT':['min', 'max', 'mean'],
        'PER_ANNUITY_CREDIT_CARD':['min', 'max', 'mean'],
        'PER_MAX_CREDIT_OVD_AMT_DEBT':['min', 'max', 'mean'],
        'PER_MAX_CREDIT_OVD_AMT_CREDIT_CARD_LIMIT':['min', 'max', 'mean'],
        'PER_MAX_CREDIT_OVD_AMT_CREDIT_LIMIT':['min', 'max', 'mean'],

        'STATUS_CLOSED': ['max', 'mean'],
        'STATUS_UNKNOWN': ['max', 'mean'],
        'DPD0': ['max', 'mean'],
        'DPD1_TO_30': ['max', 'mean'],
        'DPD31_TO_60': ['max', 'mean'],
        'DPD61_TO_90': ['max', 'mean'],
        'DPD91_TO_120': ['max', 'mean'],
        'DPD120PLUS_SOLD_WRITEOFF': ['max', 'mean'],

        'HAS_DPD': ['max', 'mean'],
        'HAS_DPD30PLUS': ['max', 'mean'],
        'HAS_DPD60PLUS': ['max', 'mean'],
        'HAS_DPD90PLUS': ['max', 'mean'],
        'HAS_DPD_OVER120': ['max', 'mean'],

        'MONTHS_BALANCE': ['min', 'max', 'mean']
      }
    bureau_grp = bureau_total.groupby('SK_ID_CURR')
    bureau_day_amt_agg = bureau_grp.agg(bureau_agg_dict)
    bureau_day_amt_agg.columns = ['BUREAU_'+('_').join(column).upper() for column in bureau_day_amt_agg.columns.ravel()]
    bureau_day_amt_agg = bureau_day_amt_agg.reset_index()
    print('bureau_day_amt_agg shape:', bureau_day_amt_agg.shape)
    return bureau_day_amt_agg

In [ ]:
def application_preprocessing(apps):
    apps = apps[apps['CODE_GENDER'] != 'XNA']

    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    apps['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
    apps['DAYS_EMPLOYED_PERC'] = apps['DAYS_EMPLOYED'] / apps['DAYS_BIRTH']
    apps['INCOME_CREDIT_PERC'] = apps['AMT_INCOME_TOTAL'] / apps['AMT_CREDIT']
    apps['INCOME_PER_PERSON'] = apps['AMT_INCOME_TOTAL'] / apps['CNT_FAM_MEMBERS']
    apps['ANNUITY_INCOME_PERC'] = apps['AMT_ANNUITY'] / (1 + apps['AMT_INCOME_TOTAL'])

    apps['APPS_CREDIT_GOODS'] = apps['AMT_CREDIT'] / apps['AMT_GOODS_PRICE']
    apps['APPS_CREDIT_MINUS_GOODS'] = apps['AMT_CREDIT'] - apps['AMT_GOODS_PRICE']
    apps['reg_div_publish'] = apps['DAYS_REGISTRATION'] / apps['DAYS_ID_PUBLISH']
    apps['birth_div_reg'] = apps['DAYS_BIRTH'] / apps['DAYS_REGISTRATION']

    apps['NEW_PHONE_TO_BIRTH_RATIO'] = apps['DAYS_LAST_PHONE_CHANGE'] / apps['DAYS_BIRTH']
    apps['NEW_PHONE_TO_BIRTH_RATIO_EMPLOYER'] = apps['DAYS_LAST_PHONE_CHANGE'] / apps['DAYS_EMPLOYED']
    
    apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
    apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())
    
    # AMT_ANNUITY - amount should be paid per month.
    # AMT_CREDIT  - total amount of loan.
    # AMT_GOODS_PRICE : consumer loadn.eg) car purchase installment.
    apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
    apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
    
    # AMT_INCOME_TOTAL : income 
    # CNT_FAM_MEMBERS  : the number of family members
    apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
    apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']
    apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']
    
    # DAYS_BIRTH : Client's age in days at the time of application
    # DAYS_EMPLOYED : How many days before the application the person started current employment
    apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
    apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
    apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
    apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
    apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']
    
    return apps

In [ ]:
def all_infomation_gathering(application_train, POS_CASH_balance, bureau, bureau_balance, credit_card_balance, installments_payments, previous_application):
    # POS_CASH_balance
    POS_CASH_balance_new = pd.merge(application_train[['SK_ID_CURR']], POS_CASH_balance, how='inner', on='SK_ID_CURR')
    POS_CASH_balance_new = POS_CASH_balance_preprocessing(POS_CASH_balance_new)
    # bureau
    bureau_new = pd.merge(application_train[['SK_ID_CURR']], bureau, how='inner', on='SK_ID_CURR')
    bureau_total = bureau_preprocessing(bureau_new, bureau_balance)
    # credit card balance
    credit_card_balance_new = pd.merge(application_train[['SK_ID_CURR']], credit_card_balance, how='inner', on='SK_ID_CURR')
    credit_card_balance_new = credit_card_balance_preprocessing(credit_card_balance)
    # installments_payments
    installments_payments_new = pd.merge(application_train[['SK_ID_CURR']], installments_payments, how='inner', on='SK_ID_CURR')
    installments_payments_new = installments_payments_preprocessing(installments_payments_new)
    # previous_application
    previous_application_new = pd.merge(application_train[['SK_ID_CURR']], previous_application, how='inner', on='SK_ID_CURR')
    previous_application_new = previous_application_preprocessing(previous_application_new)
    # application train|test
    application_train_new = application_preprocessing(application_train)
    
    # Join data
    apps_all = pd.merge(application_train_new, previous_application_new, how = 'left', on = 'SK_ID_CURR')
    apps_all = pd.merge(apps_all, installments_payments_new, how = 'left', on = 'SK_ID_CURR')
    apps_all = pd.merge(apps_all, credit_card_balance_new,  how = 'left', on = 'SK_ID_CURR')
    apps_all = pd.merge(apps_all, POS_CASH_balance_new,  how = 'left', on = 'SK_ID_CURR')
    apps_all = pd.merge(apps_all, bureau_total,  how = 'left', on = 'SK_ID_CURR')
    
    # Remove string columns
    apps_all = apps_all[apps_all.columns[apps_all.dtypes != 'object']]
    
    return apps_all

# Import data

In [ ]:
application_train = pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
application_test = pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
bureau = pd.read_csv('/kaggle/input/home-credit-default-risk/bureau.csv')
bureau_balance = pd.read_csv('/kaggle/input/home-credit-default-risk/bureau_balance.csv')
POS_CASH_balance = pd.read_csv('/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv')
credit_card_balance = pd.read_csv('/kaggle/input/home-credit-default-risk/credit_card_balance.csv')
previous_application = pd.read_csv('/kaggle/input/home-credit-default-risk/previous_application.csv')
installments_payments = pd.read_csv('/kaggle/input/home-credit-default-risk/installments_payments.csv')

## Print data structure

In [ ]:
print("application_train")
data_description(application_train)
print()
print("application_test")
data_description(application_test)
print()
print("bureau")
data_description(bureau)
print()
print("bureau_balance")
data_description(bureau_balance)
print()
print("POS_CASH_balance")
data_description(POS_CASH_balance)
print()
print("credit_card_balance")
data_description(credit_card_balance)
print()
print("previous_application")
data_description(previous_application)
print()
print("installments_payments")
data_description(installments_payments)

## Combine data

In [ ]:
apps_all_train = all_infomation_gathering(application_train, POS_CASH_balance, bureau, bureau_balance, credit_card_balance, installments_payments, previous_application)
apps_all_test = all_infomation_gathering(application_test, POS_CASH_balance, bureau, bureau_balance, credit_card_balance, installments_payments, previous_application)

## Slicing data

Extract 40% of the orginal data for training models

In [ ]:
X = apps_all_train.drop('TARGET', axis=1)
y = apps_all_train['TARGET']

# Create an instance of StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=42)

# Generate the splits and extract the stratified sample
for train_index, test_index in sss.split(X, y):
    X_strat_sample, y_strat_sample = X.iloc[test_index], y.iloc[test_index]

# Save this to output working space
apps_train = X_strat_sample.copy()
apps_train["TARGET"] = y_strat_sample

# Principal Component Analysis

In [ ]:
def PCA_data_processing(df, random_state, drop_columns_arr, target_col, PCA_var_explain, n_components, apps_all_test):
    # deal with missing data
  df = df.replace([np.inf, -np.inf], np.nan)

  ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
  target_app = df[target_col]                  # target datasets

  # Get the correlation matrix
  corr_matrix = ftr_app.corr()
  # Find the pairs of columns that are perfectly correlated
  correlated_columns = corr_matrix.where(corr_matrix >= 0.7).dropna(axis=1)
  # Remove the correlated columns
  ftr_app = ftr_app.drop(correlated_columns.columns, axis=1)

  if not correlated_columns.empty:
    # Remove the correlated columns
    ftr_app = ftr_app.drop(correlated_columns.columns, axis=1)

  # Fill NaN with median value
  ftr_app = ftr_app.fillna(-999)

  # Extract the matrix from the DataFrame
  matrix = ftr_app.values

  # Find the rank of the matrix
  rank = np.linalg.matrix_rank(matrix)

  # Perform PCA on the data
  pca = PCA()
  pca.fit(ftr_app)
  print()
  print("Explained variance ratio:")
  plt.plot(np.cumsum(pca.explained_variance_ratio_))
  plt.xlabel('number of components for red')
  plt.ylabel('cumlative explained variance')

  number_of_PCs = 0
  for i in np.cumsum(pca.explained_variance_ratio_):
    if i < PCA_var_explain: 
      number_of_PCs += 1
    if i >= PCA_var_explain:
      print(i)
      break

  # Standardize the features
  scaler = StandardScaler()
  normalized_train_dataset = scaler.fit_transform(ftr_app)
  if n_components is not None:
    pca = PCA(n_components=n_components)
  else:
    pca = PCA(n_components=number_of_PCs)
  X_pca = pca.fit_transform(normalized_train_dataset)

  # Get the transformed dataset
  X_pca = pd.DataFrame(X_pca)
  print("Size X_pca: ")
  print(X_pca.shape)
    
  X_test = apps_all_test.loc[:, apps_all_test.columns != 'SK_ID_CURR']
  
  X_test = X_test.drop(correlated_columns.columns, axis=1)

  if not correlated_columns.empty:
    # Remove the correlated columns
    X_test = X_test.drop(correlated_columns.columns, axis=1)

  # deal with missing data
  X_test = X_test.replace([np.inf, -np.inf], np.nan)
  X_test = X_test.fillna(-999)
  
  normalized_dataset_test = scaler.transform(X_test)
  X_pca_test = pca.transform(normalized_dataset_test)

  # Get the transformed dataset
  X_pca_test = pd.DataFrame(X_pca_test)
    
  return ftr_app, target_app, X_pca, X_pca_test

In [ ]:
# Save output
ftr_app, target_app, X_pca, X_pca_test = PCA_data_processing(df = apps_train, random_state = 10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', PCA_var_explain=0.99999, n_components=170, apps_all_test=apps_all_test)

# Popular ML Algorithms

## Logistic Regression with PCA

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_LogisticReg_model(ftr_app, target_app, X_pca, X_pca_test, df, nfolds, random_state, drop_columns_arr, target_col, PCA_var_explain, n_components, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']
  
  # KFold
  folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(X_pca.shape[0])
  auc_scores = []

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = X_pca.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = X_pca.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        # Fit Model
        clf.fit(train_x, train_y)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # calculate p-values
  X2 = sm.add_constant(X_pca)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]

  y_pred_test = clf.predict_proba(X_pca_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
# Save output for evaluation and submission
df_off_preds, clf, res = KFold_LogisticReg_model(ftr_app=ftr_app, target_app=target_app, X_pca=X_pca, X_pca_test=X_pca_test, df=apps_train, nfolds=5, random_state = 10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', PCA_var_explain=0.99999, n_components=170, apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_LogisticRegression_train.csv", index=False)
res.to_csv("/kaggle/working/KFold_LogisticRegression_test.csv", index=False)

In [ ]:
# Random Over Sampling
def KFold_ROS_LogisticReg_model(ftr_app, target_app, X_pca, X_pca_test, df, nfolds, random_state, drop_columns_arr, target_col, PCA_var_explain, n_components, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']
  
  # KFold
  folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(X_pca.shape[0])
  auc_scores = []
  # ROS
  ros = RandomOverSampler()

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = X_pca.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = X_pca.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)
        
        # Fit ROS
        clf.fit(train_x_resampled, train_y_resampled)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # calculate p-values
  X2 = sm.add_constant(X_pca)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]

  y_pred_test = clf.predict_proba(X_pca_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
# Save output for evaluation and submission
df_off_preds, clf, res = KFold_ROS_LogisticReg_model(ftr_app=ftr_app, target_app=target_app, X_pca=X_pca, X_pca_test=X_pca_test, df=apps_train, nfolds=5, random_state = 10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', PCA_var_explain=0.99999, n_components=170, apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_LogisticRegression_train.csv", index=False)
res.to_csv("/kaggle/working/KFold_ROS_LogisticRegression_test.csv", index=False)

In [ ]:
# Random Under Sampling
def KFold_RUS_LogisticReg_model(ftr_app, target_app, X_pca, X_pca_test, df, nfolds, random_state, drop_columns_arr, target_col, PCA_var_explain, n_components, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']
  
  # KFold
  folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(X_pca.shape[0])
  auc_scores = []
  # RUS
  rus = RandomUnderSampler(random_state = 42)

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = X_pca.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = X_pca.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)
        
        # Fit RUS
        clf.fit(train_x_resampled, train_y_resampled)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # calculate p-values
  X2 = sm.add_constant(X_pca)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]

  y_pred_test = clf.predict_proba(X_pca_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
# Save output for evaluation and submission
df_off_preds, clf, res = KFold_RUS_LogisticReg_model(ftr_app=ftr_app, target_app=target_app, X_pca=X_pca, X_pca_test=X_pca_test, df=apps_train, nfolds=5, random_state = 10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', PCA_var_explain=0.99999, n_components=170, apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_LogisticRegression_train.csv", index=False)
res.to_csv("/kaggle/working/KFold_RUS_LogisticRegression_test.csv", index=False)

In [ ]:
# SMOTE
def KFold_SMOTE_LogisticReg_model(ftr_app, target_app, X_pca, X_pca_test, df, nfolds, random_state, drop_columns_arr, target_col, PCA_var_explain, n_components, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']
  
  # KFold
  folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(X_pca.shape[0])
  auc_scores = []
  # SMOTE
  smote = SMOTE()

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = X_pca.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = X_pca.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)
        
        # Fit SMOTE
        clf.fit(train_x_resampled, train_y_resampled)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # calculate p-values
  X2 = sm.add_constant(X_pca)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]

  y_pred_test = clf.predict_proba(X_pca_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
# Save output for evaluation and submission
df_off_preds, clf, res = KFold_SMOTE_LogisticReg_model(ftr_app=ftr_app, target_app=target_app, X_pca=X_pca, X_pca_test=X_pca_test, df=apps_train, nfolds=5, random_state = 10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', PCA_var_explain=0.99999, n_components=170, apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_LogisticRegression_train.csv", index=False)
res.to_csv("/kaggle/working/KFold_SMOTE_LogisticRegression_test.csv", index=False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_LogisticReg_model(ftr_app, target_app, X_pca, X_pca_test, df, nfolds, random_state, drop_columns_arr, target_col, PCA_var_explain, n_components, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']
  
  # StratifiedKFold
  folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(X_pca.shape[0])
  auc_scores = []

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = X_pca.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = X_pca.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        # Fit Model
        clf.fit(train_x, train_y)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # calculate p-values
  X2 = sm.add_constant(X_pca)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]

  y_pred_test = clf.predict_proba(X_pca_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
# Save output for evaluation and submission
df_off_preds, clf, res = StratifiedKFold_LogisticReg_model(ftr_app=ftr_app, target_app=target_app, X_pca=X_pca, X_pca_test=X_pca_test, df=apps_train, nfolds=5, random_state = 10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', PCA_var_explain=0.99999, n_components=170, apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_LogisticRegression_train.csv", index=False)
res.to_csv("/kaggle/working/StratifiedKFold_LogisticRegression_test.csv", index=False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_LogisticReg_model(ftr_app, target_app, X_pca, X_pca_test, df, nfolds, random_state, drop_columns_arr, target_col, PCA_var_explain, n_components, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']
  
  # Stratified KFold
  folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(X_pca.shape[0])
  auc_scores = []
  # ROS
  ros = RandomOverSampler()

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = X_pca.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = X_pca.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)
        
        # Fit ROS
        clf.fit(train_x_resampled, train_y_resampled)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # calculate p-values
  X2 = sm.add_constant(X_pca)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]

  y_pred_test = clf.predict_proba(X_pca_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
# Save output for evaluation and submission
df_off_preds, clf, res = StratifiedKFold_ROS_LogisticReg_model(ftr_app=ftr_app, target_app=target_app, X_pca=X_pca, X_pca_test=X_pca_test, df=apps_train, nfolds=5, random_state = 10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', PCA_var_explain=0.99999, n_components=170, apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_LogisticRegression_train.csv", index=False)
res.to_csv("/kaggle/working/StratifiedKFold_ROS_LogisticRegression_test.csv", index=False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_LogisticReg_model(ftr_app, target_app, X_pca, X_pca_test, df, nfolds, random_state, drop_columns_arr, target_col, PCA_var_explain, n_components, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']
  
  # StratifiedKFold
  folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(X_pca.shape[0])
  auc_scores = []
  # RUS
  rus = RandomUnderSampler(random_state=42)

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = X_pca.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = X_pca.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)
        
        # Fit RUS
        clf.fit(train_x_resampled, train_y_resampled)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # calculate p-values
  X2 = sm.add_constant(X_pca)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]

  y_pred_test = clf.predict_proba(X_pca_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
# Save output for evaluation and submission
df_off_preds, clf, res = StratifiedKFold_RUS_LogisticReg_model(ftr_app=ftr_app, target_app=target_app, X_pca=X_pca, X_pca_test=X_pca_test, df=apps_train, nfolds=5, random_state = 10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', PCA_var_explain=0.99999, n_components=170, apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_LogisticRegression_train.csv", index=False)
res.to_csv("/kaggle/working/StratifiedKFold_RUS_LogisticRegression_test.csv", index=False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_LogisticReg_model(ftr_app, target_app, X_pca, X_pca_test, df, nfolds, random_state, drop_columns_arr, target_col, PCA_var_explain, n_components, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']
  
  # StratifiedKFold
  folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(X_pca.shape[0])
  auc_scores = []
  # SMOTE
  smote = SMOTE()

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = X_pca.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = X_pca.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)
        
        # Fit SMOTE
        clf.fit(train_x_resampled, train_y_resampled)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # calculate p-values
  X2 = sm.add_constant(X_pca)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]

  y_pred_test = clf.predict_proba(X_pca_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
# Save output for evaluation and submission
df_off_preds, clf, res = StratifiedKFold_SMOTE_LogisticReg_model(ftr_app=ftr_app, target_app=target_app, X_pca=X_pca, X_pca_test=X_pca_test, df=apps_train, nfolds=5, random_state = 10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', PCA_var_explain=0.99999, n_components=170, apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_LogisticRegression_train.csv", index=False)
res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_LogisticRegression_test.csv", index=False)

## $k$-Nearest Neighbors

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_kNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_kNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_kNN_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_kNN_test.csv", index=False)

In [ ]:
# Random Over Sampling
def KFold_ROS_kNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    # ROS
    ros = RandomOverSampler()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_ROS_kNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_kNN_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_ROS_kNN_test.csv", index=False)

In [ ]:
# Random Under Sampling
def KFold_RUS_kNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    # RUS
    rus = RandomUnderSampler(random_state=42)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_RUS_kNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_kNN_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_RUS_kNN_test.csv", index=False)

In [ ]:
# SMOTE
def KFold_SMOTE_kNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    # SMOTE
    smote = SMOTE()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_SMOTE_kNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_kNN_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_kNN_test.csv", index=False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_kNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_kNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_kNN_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_kNN_test.csv", index=False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_kNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    # ROS
    ros = RandomOverSampler()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_ROS_kNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_kNN_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_kNN_test.csv", index=False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_kNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    # RUS
    rus = RandomUnderSampler(random_state=42)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_RUS_kNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_kNN_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_kNN_test.csv", index=False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_kNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    # SMOTE
    smote = SMOTE()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_SMOTE_kNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_kNN_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_kNN_test.csv", index=False)

## Support Vector Machines

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_SVM_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = SVC(kernel='rbf', probability=True)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict(valid_x)

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_SVM_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_SVM_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_SVM_test.csv", index=False)

In [ ]:
# Random Under Sampling
def KFold_RUS_SVM_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = SVC(kernel='rbf', probability=True)
    rus = RandomUnderSampler(random_state=42)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:,1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)[:,1]
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_RUS_SVM_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_SVM_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_RUS_SVM_test.csv", index=False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_SVM_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = SVC(kernel='rbf', probability=True)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict(valid_x)

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_SVM_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SVM_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SVM_test.csv", index=False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_SVM_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = SVC(kernel='rbf', probability=True)
    rus = RandomUnderSampler(random_state=42)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:,1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)[:,1]
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_RUS_SVM_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_SVM_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_SVM_test.csv", index=False)

## Decision Tree

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_DecisionTree_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    # Initialize the Decision Tree model
    dt = DecisionTreeClassifier(max_depth=20, random_state=42)

    # Initialize the CalibratedClassifierCV object with the Decision Tree model and 5-fold cross-validation
    kf = KFold(n_splits=5)
    calibrated_dt = CalibratedClassifierCV(dt, cv=kf, method='isotonic')

    # Initialize the AUC score
    auc_scores = []

    # Fit the calibrated model on the data and calculate AUC on the testing data
    calibrated_dt.fit(ftr_app, target_app)

    # Predict probabilities on the testing data
    y_pred_prob = calibrated_dt.predict_proba(ftr_app)[:, 1]

    # Calculate the AUC score
    auc = roc_auc_score(target_app, y_pred_prob)
    auc_scores.append(auc)

    # Print the mean and standard deviation of the AUC scores across all folds
    print(f'Mean AUC: {np.mean(auc_scores):.4f}')

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = calibrated_dt.predict_proba(X_apps_test)[:, 1]
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, test_res

In [ ]:
# isotonic, max_dept = 20
df_off_preds, test_res = KFold_DecisionTree_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_DecisionTree_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_DecisionTree_test.csv", index = False)

In [ ]:
# Random Over Sampling
def KFold_ROS_DecisionTree_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    # ROS
    ros = RandomOverSampler()
    ftr_app, target_app = ros.fit_resample(ftr_app, target_app)
    
    # Initialize the Decision Tree model
    dt = DecisionTreeClassifier(max_depth=20, random_state=42)

    # Initialize the CalibratedClassifierCV object with the Decision Tree model and 5-fold cross-validation
    kf = KFold(n_splits=5)
    calibrated_dt = CalibratedClassifierCV(dt, cv=kf, method='isotonic')

    # Initialize the AUC score
    auc_scores = []

    # Fit the calibrated model on the data and calculate AUC on the testing data
    calibrated_dt.fit(ftr_app, target_app)

    # Predict probabilities on the testing data
    y_pred_prob = calibrated_dt.predict_proba(ftr_app)[:, 1]

    # Calculate the AUC score
    auc = roc_auc_score(target_app, y_pred_prob)
    auc_scores.append(auc)

    # Print the mean and standard deviation of the AUC scores across all folds
    print(f'Mean AUC: {np.mean(auc_scores):.4f}')

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = calibrated_dt.predict_proba(X_apps_test)[:, 1]
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, test_res

In [ ]:
# isotonic, max_dept = 20
df_off_preds, test_res = KFold_ROS_DecisionTree_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_DecisionTree_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_ROS_DecisionTree_test.csv", index = False)

In [ ]:
# Random Under Sampling
def KFold_RUS_DecisionTree_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    # RUS
    rus = RandomUnderSampler(random_state=42)
    ftr_app, target_app = rus.fit_resample(ftr_app, target_app)
    
    # Initialize the Decision Tree model
    dt = DecisionTreeClassifier(max_depth=20, random_state=42)

    # Initialize the CalibratedClassifierCV object with the Decision Tree model and 5-fold cross-validation
    kf = KFold(n_splits=5)
    calibrated_dt = CalibratedClassifierCV(dt, cv=kf, method='isotonic')

    # Initialize the AUC score
    auc_scores = []

    # Fit the calibrated model on the data and calculate AUC on the testing data
    calibrated_dt.fit(ftr_app, target_app)

    # Predict probabilities on the testing data
    y_pred_prob = calibrated_dt.predict_proba(ftr_app)[:, 1]

    # Calculate the AUC score
    auc = roc_auc_score(target_app, y_pred_prob)
    auc_scores.append(auc)

    # Print the mean and standard deviation of the AUC scores across all folds
    print(f'Mean AUC: {np.mean(auc_scores):.4f}')

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = calibrated_dt.predict_proba(X_apps_test)[:, 1]
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, test_res

In [ ]:
# isotonic, max_dept = 20
df_off_preds, test_res = KFold_RUS_DecisionTree_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_DecisionTree_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_RUS_DecisionTree_test.csv", index = False)

In [ ]:
# SMOTE
def KFold_SMOTE_DecisionTree_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    # SMOTE
    smote = SMOTE()
    ftr_app, target_app = smote.fit_resample(ftr_app, target_app)
    
    # Initialize the Decision Tree model
    dt = DecisionTreeClassifier(max_depth=20, random_state=42)

    # Initialize the CalibratedClassifierCV object with the Decision Tree model and 5-fold cross-validation
    kf = KFold(n_splits=5)
    calibrated_dt = CalibratedClassifierCV(dt, cv=kf, method='isotonic')

    # Initialize the AUC score
    auc_scores = []

    # Fit the calibrated model on the data and calculate AUC on the testing data
    calibrated_dt.fit(ftr_app, target_app)

    # Predict probabilities on the testing data
    y_pred_prob = calibrated_dt.predict_proba(ftr_app)[:, 1]

    # Calculate the AUC score
    auc = roc_auc_score(target_app, y_pred_prob)
    auc_scores.append(auc)

    # Print the mean and standard deviation of the AUC scores across all folds
    print(f'Mean AUC: {np.mean(auc_scores):.4f}')

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = calibrated_dt.predict_proba(X_apps_test)[:, 1]
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, test_res

In [ ]:
# isotonic, max_dept = 20
df_off_preds, test_res = KFold_SMOTE_DecisionTree_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_DecisionTree_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_DecisionTree_test.csv", index = False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_DecisionTree_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    # Initialize the Decision Tree model
    dt = DecisionTreeClassifier(max_depth=20, random_state=42)

    # Initialize the CalibratedClassifierCV object with the Decision Tree model and 5-fold cross-validation
    skf = StratifiedKFold(n_splits=5)
    calibrated_dt = CalibratedClassifierCV(dt, cv=skf, method='isotonic')

    # Initialize the AUC score
    auc_scores = []

    # Fit the calibrated model on the data and calculate AUC on the testing data
    calibrated_dt.fit(ftr_app, target_app)

    # Predict probabilities on the testing data
    y_pred_prob = calibrated_dt.predict_proba(ftr_app)[:, 1]

    # Calculate the AUC score
    auc = roc_auc_score(target_app, y_pred_prob)
    auc_scores.append(auc)

    # Print the mean and standard deviation of the AUC scores across all folds
    print(f'Mean AUC: {np.mean(auc_scores):.4f}')

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = calibrated_dt.predict_proba(X_apps_test)[:, 1]
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, test_res

In [ ]:
# isotonic, max_dept = 20
df_off_preds, test_res = StratifiedKFold_DecisionTree_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifedKFold_DecisionTree_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifedKFold_DecisionTree_test.csv", index = False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_DecisionTree_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    # ROS
    ros = RandomOverSampler()
    ftr_app, target_app = ros.fit_resample(ftr_app, target_app)
    
    # Initialize the Decision Tree model
    dt = DecisionTreeClassifier(max_depth=20, random_state=42)

    # Initialize the CalibratedClassifierCV object with the Decision Tree model and 5-fold cross-validation
    skf = StratifiedKFold(n_splits=5)
    calibrated_dt = CalibratedClassifierCV(dt, cv=skf, method='isotonic')

    # Initialize the AUC score
    auc_scores = []

    # Fit the calibrated model on the data and calculate AUC on the testing data
    calibrated_dt.fit(ftr_app, target_app)

    # Predict probabilities on the testing data
    y_pred_prob = calibrated_dt.predict_proba(ftr_app)[:, 1]

    # Calculate the AUC score
    auc = roc_auc_score(target_app, y_pred_prob)
    auc_scores.append(auc)

    # Print the mean and standard deviation of the AUC scores across all folds
    print(f'Mean AUC: {np.mean(auc_scores):.4f}')

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = calibrated_dt.predict_proba(X_apps_test)[:, 1]
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, test_res

In [ ]:
# isotonic, max_dept = 20
df_off_preds, test_res = StratifiedKFold_ROS_DecisionTree_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_DecisionTree_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_DecisionTree_test.csv", index = False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_DecisionTree_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    # RUS
    rus = RandomUnderSampler(random_state=42)
    ftr_app, target_app = rus.fit_resample(ftr_app, target_app)
    
    # Initialize the Decision Tree model
    dt = DecisionTreeClassifier(max_depth=20, random_state=42)

    # Initialize the CalibratedClassifierCV object with the Decision Tree model and 5-fold cross-validation
    skf = StratifiedKFold(n_splits=5)
    calibrated_dt = CalibratedClassifierCV(dt, cv=skf, method='isotonic')

    # Initialize the AUC score
    auc_scores = []

    # Fit the calibrated model on the data and calculate AUC on the testing data
    calibrated_dt.fit(ftr_app, target_app)

    # Predict probabilities on the testing data
    y_pred_prob = calibrated_dt.predict_proba(ftr_app)[:, 1]

    # Calculate the AUC score
    auc = roc_auc_score(target_app, y_pred_prob)
    auc_scores.append(auc)

    # Print the mean and standard deviation of the AUC scores across all folds
    print(f'Mean AUC: {np.mean(auc_scores):.4f}')

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = calibrated_dt.predict_proba(X_apps_test)[:, 1]
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, test_res

In [ ]:
# isotonic, max_dept = 20
df_off_preds, test_res = StratifiedKFold_RUS_DecisionTree_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_DecisionTree_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_DecisionTree_test.csv", index = False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_DecisionTree_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    # SMOTE
    smote = SMOTE()
    ftr_app, target_app = smote.fit_resample(ftr_app, target_app)
    
    # Initialize the Decision Tree model
    dt = DecisionTreeClassifier(max_depth=20, random_state=42)

    # Initialize the CalibratedClassifierCV object with the Decision Tree model and 5-fold cross-validation
    skf = StratifiedKFold(n_splits=5)
    calibrated_dt = CalibratedClassifierCV(dt, cv=skf, method='isotonic')

    # Initialize the AUC score
    auc_scores = []

    # Fit the calibrated model on the data and calculate AUC on the testing data
    calibrated_dt.fit(ftr_app, target_app)

    # Predict probabilities on the testing data
    y_pred_prob = calibrated_dt.predict_proba(ftr_app)[:, 1]

    # Calculate the AUC score
    auc = roc_auc_score(target_app, y_pred_prob)
    auc_scores.append(auc)

    # Print the mean and standard deviation of the AUC scores across all folds
    print(f'Mean AUC: {np.mean(auc_scores):.4f}')

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = calibrated_dt.predict_proba(X_apps_test)[:, 1]
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, test_res

In [ ]:
# isotonic, max_dept = 20
df_off_preds, test_res = StratifiedKFold_SMOTE_DecisionTree_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/SratifiedKFold_SMOTE_DecisionTree_train.csv", index = False)
test_res.to_csv("/kaggle/working/SratifiedKFold_SMOTE_DecisionTree_test.csv", index = False)

## Neural Networks

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_CNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)

    # Define the neural network architecture
    clf = Sequential()
    clf.add(Dense(128, input_dim=ftr_app.shape[1], activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(256, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(512, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1024, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(2048, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1, activation='softmax'))

    # Compile the model
    clf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[AUC()])
    
    auc_scores = []
    oof_preds = np.zeros(ftr_app.shape[0])
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y, epochs=1, batch_size=32, validation_data=(valid_x, valid_y))
        # Save predicted proba labels for validation set
        valid_preds = clf.predict(valid_x)
        oof_preds_proba[valid_idx] = valid_preds.ravel()
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, valid_preds.ravel())
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    # compute mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    
    # Return target of training set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
# 128 -> 2048
df_off_preds, clf, test_res = KFold_CNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_CNN_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_CNN_test.csv", index = False)

In [ ]:
# Random Over Sampling
def KFold_ROS_CNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    # ROS
    ros = RandomOverSampler()

    # Define the neural network architecture
    clf = Sequential()
    clf.add(Dense(128, input_dim=ftr_app.shape[1], activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(256, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(512, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1024, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(2048, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1, activation='softmax'))

    # Compile the model
    clf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[AUC()])
    
    auc_scores = []
    oof_preds = np.zeros(ftr_app.shape[0])
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, epochs=1, batch_size=32, validation_data=(valid_x, valid_y))
        # Save predicted proba labels for validation set
        valid_preds = clf.predict(valid_x)
        oof_preds_proba[valid_idx] = valid_preds.ravel()
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, valid_preds.ravel())
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    # compute mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    
    # Return target of training set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
# 128 -> 2048
df_off_preds, clf, test_res = KFold_ROS_CNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_CNN_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_ROS_CNN_test.csv", index = False)

In [ ]:
# Random Under Sampling
def KFold_RUS_CNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    # RUS
    rus = RandomUnderSampler(random_state =42)

    # Define the neural network architecture
    clf = Sequential()
    clf.add(Dense(128, input_dim=ftr_app.shape[1], activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(256, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(512, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1024, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(2048, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1, activation='softmax'))

    # Compile the model
    clf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[AUC()])
    
    auc_scores = []
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, epochs=1, batch_size=32, validation_data=(valid_x, valid_y))
        # Save predicted proba labels for validation set
        valid_preds = clf.predict(valid_x)
        oof_preds_proba[valid_idx] = valid_preds.ravel()
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, valid_preds.ravel())
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    # compute mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    
    # Return target of training set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
# 128 -> 2048
df_off_preds, clf, test_res = KFold_RUS_CNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_CNN_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_RUS_CNN_test.csv", index = False)

In [ ]:
# SMOTE
def KFold_SMOTE_CNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    # SMOTE
    smote = SMOTE()

    # Define the neural network architecture
    clf = Sequential()
    clf.add(Dense(128, input_dim=ftr_app.shape[1], activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(256, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(512, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1024, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(2048, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1, activation='softmax'))

    # Compile the model
    clf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[AUC()])
    
    auc_scores = []
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, epochs=1, batch_size=32, validation_data=(valid_x, valid_y))
        # Save predicted proba labels for validation set
        valid_preds = clf.predict(valid_x)
        oof_preds_proba[valid_idx] = valid_preds.ravel()
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, valid_preds.ravel())
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    # compute mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    
    # Return target of training set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
# 128 -> 2048
df_off_preds, clf, test_res = KFold_SMOTE_CNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_CNN_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_CNN_test.csv", index = False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_CNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratifed KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)

    # Define the neural network architecture
    clf = Sequential()
    clf.add(Dense(128, input_dim=ftr_app.shape[1], activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(256, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(512, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1024, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(2048, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1, activation='softmax'))

    # Compile the model
    clf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[AUC()])
    
    auc_scores = []
    oof_preds = np.zeros(ftr_app.shape[0])
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y, epochs=1, batch_size=32, validation_data=(valid_x, valid_y))
        # Save predicted proba labels for validation set
        valid_preds = clf.predict(valid_x)
        oof_preds_proba[valid_idx] = valid_preds.ravel()
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, valid_preds.ravel())
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    # compute mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    
    # Return target of training set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
# 128 -> 2048
df_off_preds, clf, test_res = StratifiedKFold_CNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_CNN_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_CNN_test.csv", index = False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_CNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    # ROS
    ros = RandomOverSampler()

    # Define the neural network architecture
    clf = Sequential()
    clf.add(Dense(128, input_dim=ftr_app.shape[1], activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(256, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(512, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1024, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(2048, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1, activation='softmax'))

    # Compile the model
    clf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[AUC()])
    
    auc_scores = []
    oof_preds = np.zeros(ftr_app.shape[0])
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, epochs=1, batch_size=32, validation_data=(valid_x, valid_y))
        # Save predicted proba labels for validation set
        valid_preds = clf.predict(valid_x)
        oof_preds_proba[valid_idx] = valid_preds.ravel()
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, valid_preds.ravel())
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    # compute mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    
    # Return target of training set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
# 128 -> 2048
df_off_preds, clf, test_res = StratifiedKFold_ROS_CNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_CNN_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_CNN_test.csv", index = False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_CNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratifed KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    # RUS
    rus = RandomUnderSampler(random_state =42)

    # Define the neural network architecture
    clf = Sequential()
    clf.add(Dense(128, input_dim=ftr_app.shape[1], activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(256, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(512, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1024, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(2048, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1, activation='softmax'))

    # Compile the model
    clf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[AUC()])
    
    auc_scores = []
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, epochs=1, batch_size=32, validation_data=(valid_x, valid_y))
        # Save predicted proba labels for validation set
        valid_preds = clf.predict(valid_x)
        oof_preds_proba[valid_idx] = valid_preds.ravel()
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, valid_preds.ravel())
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    # compute mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    
    # Return target of training set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
# 128 -> 2048
df_off_preds, clf, test_res = StratifiedKFold_RUS_CNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_CNN_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_CNN_test.csv", index = False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_CNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFolds
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    # SMOTE
    smote = SMOTE()

    # Define the neural network architecture
    clf = Sequential()
    clf.add(Dense(128, input_dim=ftr_app.shape[1], activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(256, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(512, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1024, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(2048, activation='sigmoid'))
    clf.add(Dropout(0.2))
    clf.add(Dense(1, activation='softmax'))

    # Compile the model
    clf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[AUC()])
    
    auc_scores = []
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, epochs=1, batch_size=32, validation_data=(valid_x, valid_y))
        # Save predicted proba labels for validation set
        valid_preds = clf.predict(valid_x)
        oof_preds_proba[valid_idx] = valid_preds.ravel()
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, valid_preds.ravel())
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    # compute mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    
    # Return target of training set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res[f"TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
# 128 -> 2048
df_off_preds, clf, test_res = StratifiedKFold_SMOTE_CNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_CNN_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_CNN_test.csv", index = False)

# Bagging Algorithms

## Random Forest

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_RandomForest_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_RandomForest_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_RandomForest_test.csv", index=False)

In [ ]:
# Random Over Sampling
def KFold_ROS_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    ros = RandomOverSampler()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_ROS_RandomForest_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_RandomForest_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_ROS_RandomForest_test.csv", index=False)

In [ ]:
# Random Under Sampling
def KFold_RUS_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    rus = RandomUnderSampler(random_state=42)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_RUS_RandomForest_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_RandomForest_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_RUS_RandomForest_test.csv", index=False)

In [ ]:
# SMOTE
def KFold_SMOTE_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    smote = SMOTE()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_SMOTE_RandomForest_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_RandomForest_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_RandomForest_test.csv", index=False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_RandomForest_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RandomForest_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RandomForest_test.csv", index=False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    ros = RandomOverSampler()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_ROS_RandomForest_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_RandomForest_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_RandomForest_test.csv", index=False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    rus = RandomUnderSampler(random_state=42)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_RUS_RandomForest_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_RandomForest_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_RandomForest_test.csv", index=False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    smote = SMOTE()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_SMOTE_RandomForest_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_RandomForest.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_RandomForest_test.csv", index=False)

## Bagged Neural Networks

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_MLP_CNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)

    # Define the neural network architecture
    nn = MLPClassifier(hidden_layer_sizes=(256, 128, 64, 32), activation='logistic', solver='adam', max_iter=200, early_stopping=True, verbose=500, learning_rate_init=0.008)

    # Define the bagging classifier and fit the model
    clf = BaggingClassifier(base_estimator=nn, n_estimators=100, bootstrap=True, random_state=42,verbose=500)
    
    auc_scores = []
    oof_preds_proba = np.zeros((len(df), 2))

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        valid_preds = clf.predict_proba(valid_x)
        valid_preds_reshaped = valid_preds.reshape(-1, 2)
        oof_preds_proba[valid_idx] = valid_preds_reshaped
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, valid_preds[:,1])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    # compute mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    
    # Return target of training set
    df_off_preds['TARGET'] = oof_preds_proba[:,1]
    
    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)
    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:,1]

    return df_off_preds, clf, test_res

In [ ]:
# 128 -> 2048
df_off_preds, clf, test_res = KFold_MLP_CNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_MLP_CNN_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_MLP_CNN_test.csv", index = False)

In [ ]:
# Random Under Sampling
def KFold_RUS_MLP_CNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    # RUS
    rus = RandomUnderSampler()

    # Define the neural network architecture
    nn = MLPClassifier(hidden_layer_sizes=(256, 128, 64, 32), activation='logistic', solver='adam', max_iter=200, early_stopping=True, verbose=500, learning_rate_init=0.008)

    # Define the bagging classifier and fit the model
    clf = BaggingClassifier(base_estimator=nn, n_estimators=100, bootstrap=True, random_state=42,verbose=500)
    
    auc_scores = []
    oof_preds_proba = np.zeros((len(df), 2))

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        valid_preds = clf.predict_proba(valid_x)
        valid_preds_reshaped = valid_preds.reshape(-1, 2)
        oof_preds_proba[valid_idx] = valid_preds_reshaped
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, valid_preds[:,1])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    # compute mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    
    # Return target of training set
    df_off_preds['TARGET'] = oof_preds_proba[:,1]
    
    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)
    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:,1]

    return df_off_preds, clf, test_res

In [ ]:
# 128 -> 2048
df_off_preds, clf, test_res = KFold_RUS_MLP_CNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_MLP_CNN_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_RUS_MLP_CNN_test.csv", index = False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_MLP_CNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)

    # Define the neural network architecture
    nn = MLPClassifier(hidden_layer_sizes=(256, 128, 64, 32), activation='logistic', solver='adam', max_iter=200, early_stopping=True, verbose=500, learning_rate_init=0.008)

    # Define the bagging classifier and fit the model
    clf = BaggingClassifier(base_estimator=nn, n_estimators=100, bootstrap=True, random_state=42,verbose=500)
    
    auc_scores = []
    oof_preds_proba = np.zeros((len(df), 2))

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        valid_preds = clf.predict_proba(valid_x)
        valid_preds_reshaped = valid_preds.reshape(-1, 2)
        oof_preds_proba[valid_idx] = valid_preds_reshaped
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, valid_preds[:,1])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    # compute mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    
    # Return target of training set
    df_off_preds['TARGET'] = oof_preds_proba[:,1]
    
    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)
    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:,1]

    return df_off_preds, clf, test_res

In [ ]:
# 128 -> 2048
df_off_preds, clf, test_res = StratifiedKFold_MLP_CNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_MLP_CNN_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_MLP_CNN_test.csv", index = False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_MLP_CNN_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    # RUS
    rus = RandomUnderSampler()

    # Define the neural network architecture
    nn = MLPClassifier(hidden_layer_sizes=(256, 128, 64, 32), activation='logistic', solver='adam', max_iter=200, early_stopping=True, verbose=500, learning_rate_init=0.008)

    # Define the bagging classifier and fit the model
    clf = BaggingClassifier(base_estimator=nn, n_estimators=100, bootstrap=True, random_state=42,verbose=500)
    
    auc_scores = []
    oof_preds_proba = np.zeros((len(df), 2))

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        valid_preds = clf.predict_proba(valid_x)
        valid_preds_reshaped = valid_preds.reshape(-1, 2)
        oof_preds_proba[valid_idx] = valid_preds_reshaped
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, valid_preds[:,1])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    # compute mean AUC across all folds
    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc}")
    
    # Return target of training set
    df_off_preds['TARGET'] = oof_preds_proba[:,1]
    
    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)
    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:,1]

    return df_off_preds, clf, test_res

In [ ]:
# 128 -> 2048
df_off_preds, clf, test_res = StratifiedKFold_RUS_MLP_CNN_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_MLP_CNN_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_MLP_CNN_test.csv", index = False)

# Boosting Algorithms

## AdaBoost

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_AdaBoost_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    clf = AdaBoostClassifier(
                              DecisionTreeClassifier(max_depth=5), n_estimators=1000,
                              algorithm="SAMME.R", learning_rate=0.08
    )
    
    oof_preds = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
# all data no dropping columns
df_off_preds, clf, test_res = KFold_AdaBoost_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_AdaBoost_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_AdaBoost_test.csv", index=False)

In [ ]:
# Random Over Sampling
def KFold_ROS_AdaBoost_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    ros = RandomOverSampler()
    clf = AdaBoostClassifier(
                              DecisionTreeClassifier(max_depth=5), n_estimators=200,
                              algorithm="SAMME.R", learning_rate=0.08
    )
    
    oof_preds = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)
        
        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res["TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_ROS_AdaBoost_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_AdaBoost_WL_DT_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_ROS_AdaBoost_WL_DT_test.csv", index=False)

In [ ]:
# Random Under Sampling
def KFold_RUS_AdaBoost_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    rus = RandomUnderSampler()
    clf = AdaBoostClassifier(
                              DecisionTreeClassifier(max_depth=5), n_estimators=200,
                              algorithm="SAMME.R", learning_rate=0.08
    )
    
    oof_preds = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)
        
        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res["TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_RUS_AdaBoost_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_AdaBoost_WL_DT_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_RUS_AdaBoost_WL_DT_test.csv", index=False)

In [ ]:
# SMOTE
def KFold_SMOTE_AdaBoost_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    smote = SMOTE()
    clf = AdaBoostClassifier(
                              DecisionTreeClassifier(max_depth=5), n_estimators=200,
                              algorithm="SAMME.R", learning_rate=0.08
    )
    
    oof_preds = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)
        
        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res["TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_SMOTE_AdaBoost_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_AdaBoost_WL_DT_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_AdaBoost_WL_DT_test.csv", index=False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_AdaBoost_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply StratifiedKFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    clf = AdaBoostClassifier(
                              DecisionTreeClassifier(max_depth=5), n_estimators=200,
                              algorithm="SAMME.R", learning_rate=0.08
    )
    
    oof_preds = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res["TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_AdaBoost_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_AdaBoost_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_AdaBoost_test.csv", index=False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_AdaBoost_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dataset
    target_app = df[target_col]                  # target dataset
    # apply StratifiedKFold
    folds = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=random_state)
    
    ros = RandomOverSampler()
    
    clf = AdaBoostClassifier(
                              DecisionTreeClassifier(max_depth=10), n_estimators=1000,
                              algorithm="SAMME.R", learning_rate=0.08
          )
    
    auc_scores = []
    oof_preds = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print(f"Fold {fold_idx+1} starts")
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score:.5f}")

    mean_auc = np.mean(auc_scores)
    print(f"Mean AUC: {mean_auc:.5f}")
    
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"] = df_test["SK_ID_CURR"]
    # replace infinity values with a large negative number
    # deal with missing data
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
# all data no dropping columns
df_off_preds, clf, test_res = StratifiedKFold_ROS_AdaBoost_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_AdaBoost_WL_DT_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_AdaBoost_WL_DT_test.csv", index=False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_AdaBoost_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply StratifiedKFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    rus = RandomUnderSampler()
    clf = AdaBoostClassifier(
                              DecisionTreeClassifier(max_depth=5), n_estimators=200,
                              algorithm="SAMME.R", learning_rate=0.08
    )
    
    oof_preds = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)
        
        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res["TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_RUS_AdaBoost_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_AdaBoost_WL_DT_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_AdaBoost_WL_DT_test.csv", index=False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_AdaBoost_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply StratifiedKFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    smote = SMOTE()
    clf = AdaBoostClassifier(
                              DecisionTreeClassifier(max_depth=5), n_estimators=200,
                              algorithm="SAMME.R", learning_rate=0.08
    )
    
    oof_preds = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)
        
        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict(X_apps_test)
    test_res["TARGET"] = y_apps_test

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_SMOTE_AdaBoost_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_AdaBoost_WL_DT_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_AdaBoost_WL_DT_test.csv", index=False)

## Gradient Boosting

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_GBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_GBoosting_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_GBoosting_test.csv", index=False)

In [ ]:
# Random Over Sampling
def KFold_ROS_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    ros = RandomOverSampler()
    
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_ROS_GBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_GBoosting_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_ROS_GBoosting_test.csv", index=False)

In [ ]:
# Random Under Sampling
def KFold_RUS_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    rus = RandomUnderSampler(random_state = 42)
    
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_RUS_GBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_GBoosting_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_RUS_GBoosting_test.csv", index=False)

In [ ]:
# SMOTE
def KFold_SMOTE_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    smote = SMOTE()
    
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_SMOTE_GBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_GBoosting_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_GBoosting_test.csv", index=False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_GBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_GBoosting_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_GBoosting_test.csv", index=False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply StratifiedKFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    ros = RandomOverSampler()
    
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
# all data no dropping columns
df_off_preds, clf, test_res = StratifiedKFold_ROS_GBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_GBoosting_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_GBoosting_test.csv", index=False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply StratifiedKFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    rus = RandomUnderSampler(random_state = 42)
    
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_RUS_GBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_GBoosting_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_GBoosting_test.csv", index=False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    smote = SMOTE()
    
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_SMOTE_GBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_GBoosting_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_GBoosting_test.csv", index=False)

## XGBoost

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_XGBoosting_model(df, nfolds, cutoff_threshold, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    print("AUC:")
    oof_preds = np.zeros(ftr_app.shape[0])
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    # Create an empty dictionary to store the feature importance scores
    importances = {}

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res, importances

In [ ]:
df_off_preds, clf, test_res, importances = KFold_XGBoosting_model(df=apps_train, nfolds=5, cutoff_threshold = 0.3, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_XGBoost_train.csv", index=False)
importances.to_csv("/kaggle/working/KFold_XGBoost_test.csv", index=False)

In [ ]:
# Random Over Sampling
def KFold_ROS_XGBoosting_model(df, nfolds, cutoff_threshold, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # appl KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    ros = RandomOverSampler()
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    print("AUC:")
    oof_preds = np.zeros(ftr_app.shape[0])
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_ROS_XGBoosting_model(df=apps_train, nfolds=5, cutoff_threshold = 0.3, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_XGBoost_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_ROS_XGBoost_test.csv", index=False)

In [ ]:
# Random Under Sampling
def KFold_RUS_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply StratifiedKFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    rus = RandomUnderSampler(random_state = 42)
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds = np.zeros(ftr_app.shape[0])
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_RUS_XGBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_XGBoost_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_RUS_XGBoost_test.csv", index=False)

In [ ]:
# SMOTE
def KFold_SMOTE_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    smote = SMOTE()
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds = np.zeros(ftr_app.shape[0])
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_SMOTE_XGBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_XGBoost_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_XGBoost_test.csv", index=False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_XGBoosting_model(df, nfolds, cutoff_threshold, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply StratifiedKFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    print("AUC:")
    oof_preds = np.zeros(ftr_app.shape[0])
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    # Create an empty dictionary to store the feature importance scores
    importances = {}

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res, importances

In [ ]:
df_off_preds, clf, test_res, importances = StratifiedKFold_XGBoosting_model(df=apps_train, nfolds=5, cutoff_threshold = 0.3, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_XGBoost_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_XGBoost_test.csv", index=False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    ros = RandomOverSampler()
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds = np.zeros(ftr_app.shape[0])
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_ROS_XGBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_XGBoost_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_XGBoost_test.csv", index=False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    rus = RandomUnderSampler(random_state = 42)
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds = np.zeros(ftr_app.shape[0])
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_RUS_XGBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_XGBoost_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_XGBoost_test.csv", index=False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Stratified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    smote = SMOTE()
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds = np.zeros(ftr_app.shape[0])
    oof_preds_proba = np.zeros(ftr_app.shape[0])

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted class labels for validation set
        oof_preds[valid_idx] = clf.predict(valid_x)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_SMOTE_XGBoosting_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_XGBoost_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_XGBoost_test.csv", index=False)

## Light GBM

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=100000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    test_res["SK_ID_CURR"] = df_test["SK_ID_CURR"]
    # save SK_ID_CURR
    # test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = LightGBMC_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_LightGBM_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_LightGBM_test.csv", index = False)

In [ ]:
# Random Over Sampling
def KFold_ROS_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    # ROS
    ros = RandomOverSampler()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_ROS_LightGBMC_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_LightGBM_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_ROS_LightGBM_test.csv", index=False)

In [ ]:
# Random Under Sampling
def KFold_RUS_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    # Apply RUS to the training set
    rus = RandomUnderSampler(random_state=42)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_RUS_LightGBMC_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_LightGBM_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_RUS_LightGBM_test.csv", index=False)

In [ ]:
# SMOTE
def KFold_SMOTE_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFold
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    # SMOTE
    smote = SMOTE()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_SMOTE_LightGBMC_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_LightGBM_train.csv", index=False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_LightGBM_test.csv", index=False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=100000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    test_res["SK_ID_CURR"] = df_test["SK_ID_CURR"]
    # save SK_ID_CURR
    # test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_LightGBMC_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_LightGBM_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_LightGBM_test.csv", index=False)

In [ ]:
# Random Over Sampling
def StraifiedKFold_ROS_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply Straified KFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    # ROS
    ros = RandomOverSampler()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StraifiedKFold_ROS_LightGBMC_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_LightGBM_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_LightGBM_test.csv", index=False)

In [ ]:
# Random Under Sampling
def StraifiedKFold_RUS_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply StraifiedKFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    # Apply RUS to the training set
    rus = RandomUnderSampler(random_state=42)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StraifiedKFold_RUS_LightGBMC_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_LightGBM_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_LightGBM_test.csv", index=False)

In [ ]:
# SMOTE
def StraifiedKFold_SMOTE_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)
    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply StraifiedKFold
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    # SMOTE
    smote = SMOTE()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StraifiedKFold_SMOTE_LightGBMC_model(df=apps_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=apps_all_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_LightGBM_train.csv", index=False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_LightGBM_test.csv", index=False)

# Stacking

## Import dataframe (Top 8 AUC model results)

### Training data

In [ ]:
KFold_LightGBM_train = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/KFold_LightGBM_train.csv")
KFold_LightGBM_train.rename(columns = {'TARGET':'KFold_LightGBM'}, inplace = True)
KFold_ROS_XGBoost_train = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/KFold_ROS_XGBoost_train.csv")
KFold_ROS_XGBoost_train.rename(columns = {'TARGET':'KFold_ROS_XGBoost'}, inplace = True)
KFold_RUS_XGBoost_train = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/KFold_RUS_XGBoost_train.csv")
KFold_RUS_XGBoost_train.rename(columns = {'TARGET':'KFold_RUS_XGBoost'}, inplace = True)
KFold_SMOTE_XGBoost_train = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/KFold_SMOTE_XGBoost_train.csv")
KFold_SMOTE_XGBoost_train.rename(columns = {'TARGET':'KFold_SMOTE_XGBoost'}, inplace = True)
StratifiedKFold_LightGBM_train = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/StratifiedKFold_LightGBM_train.csv")
StratifiedKFold_LightGBM_train.rename(columns = {'TARGET':'StratifiedKFold_LightGBM'}, inplace = True)
StratifiedKFold_ROS_XGBoost_train = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/StratifiedKFold_ROS_XGBoost_train.csv")
StratifiedKFold_ROS_XGBoost_train.rename(columns = {'TARGET':'StratifiedKFold_ROS_XGBoost'}, inplace = True)
StratifiedKFold_RUS_XGBoost_train = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/StratifiedKFold_RUS_XGBoost_train.csv")
StratifiedKFold_RUS_XGBoost_train.rename(columns = {'TARGET':'StratifiedKFold_RUS_XGBoost'}, inplace = True)
StratifiedKFold_SMOTE_XGBoost_train = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/StratifiedKFold_SMOTE_XGBoost_train.csv")
StratifiedKFold_SMOTE_XGBoost_train.rename(columns = {'TARGET':'StratifiedKFold_SMOTE_XGBoost'}, inplace = True)

# Merge the datasets using a common key column
merged_df_train = pd.merge(KFold_LightGBM_train, KFold_ROS_XGBoost_train, how = "inner", on='SK_ID_CURR')
merged_df_train = pd.merge(merged_df_train, KFold_RUS_XGBoost_train, how = "inner", on='SK_ID_CURR')
merged_df_train = pd.merge(merged_df_train, KFold_SMOTE_XGBoost_train, how = "inner", on='SK_ID_CURR')
merged_df_train = pd.merge(merged_df_train, StratifiedKFold_LightGBM_train, how = "inner", on='SK_ID_CURR')
merged_df_train = pd.merge(merged_df_train, StratifiedKFold_ROS_XGBoost_train, how = "inner", on='SK_ID_CURR')
merged_df_train = pd.merge(merged_df_train, StratifiedKFold_RUS_XGBoost_train, how = "inner", on='SK_ID_CURR')
merged_df_train = pd.merge(merged_df_train, StratifiedKFold_SMOTE_XGBoost_train, how = "inner", on='SK_ID_CURR')

merged_df_train = pd.merge(merged_df_train, apps_train[["SK_ID_CURR", "TARGET"]], how = "inner", on='SK_ID_CURR')

print(len(merged_df_train))
print()
merged_df_train.head()

### Testing data

In [ ]:
KFold_LightGBM_test = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/KFold_LightGBM_test.csv")
KFold_LightGBM_test.rename(columns = {'TARGET':'KFold_LightGBM'}, inplace = True)
KFold_ROS_XGBoost_test = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/KFold_ROS_XGBoost_test.csv")
KFold_ROS_XGBoost_test.rename(columns = {'TARGET':'KFold_ROS_XGBoost'}, inplace = True)
KFold_RUS_XGBoost_test = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/KFold_RUS_XGBoost_test.csv")
KFold_RUS_XGBoost_test.rename(columns = {'TARGET':'KFold_RUS_XGBoost'}, inplace = True)
KFold_SMOTE_XGBoost_test = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/KFold_SMOTE_XGBoost_test.csv")
KFold_SMOTE_XGBoost_test.rename(columns = {'TARGET':'KFold_SMOTE_XGBoost'}, inplace = True)
StratifiedKFold_LightGBM_test = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/StratifiedKFold_LightGBM_test.csv")
StratifiedKFold_LightGBM_test.rename(columns = {'TARGET':'StratifiedKFold_LightGBM'}, inplace = True)
StratifiedKFold_ROS_XGBoost_test = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/StratifiedKFold_ROS_XGBoost_test.csv")
StratifiedKFold_ROS_XGBoost_test.rename(columns = {'TARGET':'StratifiedKFold_ROS_XGBoost'}, inplace = True)
StratifiedKFold_RUS_XGBoost_test = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/StratifiedKFold_RUS_XGBoost_test.csv")
StratifiedKFold_RUS_XGBoost_test.rename(columns = {'TARGET':'StratifiedKFold_RUS_XGBoost'}, inplace = True)
StratifiedKFold_SMOTE_XGBoost_test = pd.read_csv("/kaggle/input/home-credit-default-risk-apps-all-full/StratifiedKFold_SMOTE_XGBoost_test.csv")
StratifiedKFold_SMOTE_XGBoost_test.rename(columns = {'TARGET':'StratifiedKFold_SMOTE_XGBoost'}, inplace = True)

# Merge the datasets using a common key column
merged_df_test = pd.merge(KFold_LightGBM_test, KFold_ROS_XGBoost_test, how = "inner", on='SK_ID_CURR')
merged_df_test = pd.merge(merged_df_test, KFold_RUS_XGBoost_test, how = "inner", on='SK_ID_CURR')
merged_df_test = pd.merge(merged_df_test, KFold_SMOTE_XGBoost_test, how = "inner", on='SK_ID_CURR')
merged_df_test = pd.merge(merged_df_test, StratifiedKFold_LightGBM_test, how = "inner", on='SK_ID_CURR')
merged_df_test = pd.merge(merged_df_test, StratifiedKFold_ROS_XGBoost_test, how = "inner", on='SK_ID_CURR')
merged_df_test = pd.merge(merged_df_test, StratifiedKFold_RUS_XGBoost_test, how = "inner", on='SK_ID_CURR')
merged_df_test = pd.merge(merged_df_test, StratifiedKFold_SMOTE_XGBoost_test, how = "inner", on='SK_ID_CURR')

print(len(merged_df_test))
print()
merged_df_test.head()

## Logistic regression

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_Stacking_LogisticReg_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

  ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
  target_app = df[target_col]                  # target datasets
  
  # KFold
  folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(ftr_app.shape[0])
  auc_scores = []

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        clf.fit(train_x, train_y)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # SUMMARY
  # calculate p-values
  X2 = sm.add_constant(ftr_app)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]
    
  # apps_all_test 
  df_test = apps_all_test.copy()
  X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

  y_pred_test = clf.predict_proba(X_apps_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
df_off_preds, clf, test_res = KFold_Stacking_LogisticReg_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_Stacking_LogisticReg_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_Stacking_LogisticReg_test.csv", index = False)

In [ ]:
# Random Over Sampling
def KFold_ROS_Stacking_LogisticReg_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

  ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
  target_app = df[target_col]                  # target datasets
  
  # KFold
  folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(ftr_app.shape[0])
  auc_scores = []
    
  # ROS
  ros = RandomOverSampler()

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)
        
        # Fit ROS
        clf.fit(train_x_resampled, train_y_resampled)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # SUMMARY
  # calculate p-values
  X2 = sm.add_constant(ftr_app)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]
    
  # apps_all_test 
  df_test = apps_all_test.copy()
  X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

  y_pred_test = clf.predict_proba(X_apps_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
df_off_preds, clf, test_res = KFold_ROS_Stacking_LogisticReg_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_Stacking_LogisticReg_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_ROS_Stacking_LogisticReg_test.csv", index = False)

In [ ]:
# Random Under Sampling
def KFold_RUS_Stacking_LogisticReg_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

  ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
  target_app = df[target_col]                  # target datasets
  
  # KFold
  folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(ftr_app.shape[0])
  auc_scores = []
  
  # RUS
  rus = RandomUnderSampler()

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)
        
        # Fit ROS
        clf.fit(train_x_resampled, train_y_resampled)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # SUMMARY
  # calculate p-values
  X2 = sm.add_constant(ftr_app)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]
    
  # apps_all_test 
  df_test = apps_all_test.copy()
  X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

  y_pred_test = clf.predict_proba(X_apps_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
df_off_preds, clf, test_res = KFold_RUS_Stacking_LogisticReg_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_Stacking_LogisticReg_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_RUS_Stacking_LogisticReg_test.csv", index = False)

In [ ]:
# SMOTE
def KFold_SMOTE_Stacking_LogisticReg_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

  ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
  target_app = df[target_col]                  # target datasets
  
  # KFold
  folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(ftr_app.shape[0])
  auc_scores = []
    
  # SMOTE
  smote = SMOTE()

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)
        
        # Fit ROS
        clf.fit(train_x_resampled, train_y_resampled)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # SUMMARY
  # calculate p-values
  X2 = sm.add_constant(ftr_app)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]
    
  # apps_all_test 
  df_test = apps_all_test.copy()
  X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

  y_pred_test = clf.predict_proba(X_apps_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
df_off_preds, clf, test_res = KFold_SMOTE_Stacking_LogisticReg_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_Stacking_LogisticReg_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_Stacking_LogisticReg_test.csv", index = False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_Stacking_LogisticReg_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

  ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
  target_app = df[target_col]                  # target datasets
  
  folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(ftr_app.shape[0])
  auc_scores = []

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        clf.fit(train_x, train_y)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # SUMMARY
  # calculate p-values
  X2 = sm.add_constant(ftr_app)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]
    
  # apps_all_test 
  df_test = apps_all_test.copy()
  X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

  y_pred_test = clf.predict_proba(X_apps_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_Stacking_LogisticReg_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_Stacking_LogisticReg_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_Stacking_LogisticReg_test.csv", index = False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_Stacking_LogisticReg_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

  ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
  target_app = df[target_col]                  # target datasets
  
  # Stratified KFold
  folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(ftr_app.shape[0])
  auc_scores = []
    
  # ROS
  ros = RandomOverSampler()

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)
        
        # Fit ROS
        clf.fit(train_x_resampled, train_y_resampled)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # SUMMARY
  # calculate p-values
  X2 = sm.add_constant(ftr_app)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]
    
  # apps_all_test 
  df_test = apps_all_test.copy()
  X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

  y_pred_test = clf.predict_proba(X_apps_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_ROS_Stacking_LogisticReg_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_Stacking_LogisticReg_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_Stacking_LogisticReg_test.csv", index = False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_Stacking_LogisticReg_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

  ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
  target_app = df[target_col]                  # target datasets
  
  # StratifiedKFold
  folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(ftr_app.shape[0])
  auc_scores = []
  
  # RUS
  rus = RandomUnderSampler()

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)
        
        # Fit ROS
        clf.fit(train_x_resampled, train_y_resampled)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # SUMMARY
  # calculate p-values
  X2 = sm.add_constant(ftr_app)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]
    
  # apps_all_test 
  df_test = apps_all_test.copy()
  X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

  y_pred_test = clf.predict_proba(X_apps_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_RUS_Stacking_LogisticReg_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_Stacking_LogisticReg_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_Stacking_LogisticReg_test.csv", index = False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_Stacking_LogisticReg_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
  # Data Frame
  df_off_preds = pd.DataFrame()
  df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

  ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
  target_app = df[target_col]                  # target datasets
  
  # KFold
  folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)

  clf = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                           intercept_scaling=1, max_iter=5000, multi_class='ovr', n_jobs=1,
                           penalty='l1', random_state=None, solver='saga', tol=0.0001,
                           verbose=0, warm_start=False)
  
  oof_preds_proba = np.zeros(ftr_app.shape[0])
  auc_scores = []
    
  # SMOTE
  smote = SMOTE()

  for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)
        
        # Fit ROS
        clf.fit(train_x_resampled, train_y_resampled)

        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

  print(f"Mean AUC: {np.mean(auc_scores):.5f}")  

  df_off_preds['TARGET'] = oof_preds_proba

  # SUMMARY
  # calculate p-values
  X2 = sm.add_constant(ftr_app)
  target_app_arr = np.array(target_app)
  est = sm.Logit(target_app_arr, X2)
  est2 = est.fit()
  print(est2.summary())

  # app test 
  res = pd.DataFrame()
  res["SK_ID_CURR"] = apps_all_test["SK_ID_CURR"]
    
  # apps_all_test 
  df_test = apps_all_test.copy()
  X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

  y_pred_test = clf.predict_proba(X_apps_test)[:, 1]
  res["TARGET"] = y_pred_test
  
  return df_off_preds, clf, res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_SMOTE_Stacking_LogisticReg_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_Stacking_LogisticReg_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_Stacking_LogisticReg_test.csv", index = False)

## Random Forest

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_Stacking_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_Stacking_RandomForest_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_Stacking_RandomForest_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_Stacking_RandomForest_test.csv", index = False)

In [ ]:
# Random Over Sampling
def KFold_ROS_Stacking_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    ros = RandomOverSampler()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_ROS_Stacking_RandomForest_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_Stacking_RandomForest_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_ROS_Stacking_RandomForest_test.csv", index = False)

In [ ]:
# Random Under Sampling
def KFold_RUS_Stacking_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    rus = RandomUnderSampler()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_RUS_Stacking_RandomForest_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_Stacking_RandomForest_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_RUS_Stacking_RandomForest_test.csv", index = False)

In [ ]:
# SMOTE
def KFold_SMOTE_Stacking_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    # apply KFolds
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    smote = SMOTE()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_SMOTE_Stacking_RandomForest_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_Stacking_RandomForest_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_Stacking_RandomForest_test.csv", index = False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_Stacking_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets

    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_Stacking_RandomForest_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_Stacking_RandomForest_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_Stacking_RandomForest_test.csv", index = False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_Stacking_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets

    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    ros = RandomOverSampler()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_ROS_Stacking_RandomForest_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_Stacking_RandomForest_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_Stacking_RandomForest_test.csv", index = False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_Stacking_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    rus = RandomUnderSampler()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_RUS_Stacking_RandomForest_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_Stacking_RandomForest_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_Stacking_RandomForest_test.csv", index = False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_Stacking_RandomForest_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    # deal with missing data
    df = df.replace([np.inf, -np.inf], np.nan)
    # Fill NaN with median value
    df = df.fillna(-999)

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    clf = RandomForestClassifier(n_estimators=1000
           , criterion='gini'
           , max_depth=10
           , min_samples_split=10
           , min_samples_leaf=30
           )
    
    smote = SMOTE()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]

        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")

    df_off_preds['KFold_RF_AUC_PROBA'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    # deal with missing value
    df_test = df_test.replace([np.inf, -np.inf], np.nan)
    # fill NaN with median value
    df_test = df_test.fillna(-999)
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_SMOTE_Stacking_RandomForest_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_Stacking_RandomForest_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_Stacking_RandomForest_test.csv", index = False)

## Gradient Boosting

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_Stacking_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_Stacking_GBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_Stacking_GBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_Stacking_GBoosting_test.csv", index = False)

In [ ]:
# Random Over Sampling
def KFold_ROS_Stacking_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    ros = RandomOverSampler()
    
    clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_ROS_Stacking_GBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_Stacking_GBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_ROS_Stacking_GBoosting_test.csv", index = False)

In [ ]:
# Random Under Sampling
def KFold_RUS_Stacking_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    rus = RandomUnderSampler()
    
    clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_RUS_Stacking_GBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_Stacking_GBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_RUS_Stacking_GBoosting_test.csv", index = False)

In [ ]:
# SMOTE
def KFold_SMOTE_Stacking_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    smote = SMOTE()
    
    clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_SMOTE_Stacking_GBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_Stacking_GBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_Stacking_GBoosting_test.csv", index = False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_Stacking_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        clf.fit(train_x, train_y)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_Stacking_GBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_Stacking_GBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_Stacking_GBoosting_test.csv", index = False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_Stacking_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    ros = RandomOverSampler()
    
    clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_ROS_Stacking_GBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_Stacking_GBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_Stacking_GBoosting_test.csv", index = False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_Stacking_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    rus = RandomUnderSampler()
    
    clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_RUS_Stacking_GBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_Stacking_GBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_Stacking_GBoosting_test.csv", index = False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_Stacking_GBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    smote = SMOTE()
    
    clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.008, max_depth=5, random_state=0, loss='deviance')
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    auc_scores = []

    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
        # compute AUC on validation set
        auc_score = roc_auc_score(valid_y, oof_preds_proba[valid_idx])
        auc_scores.append(auc_score)
        print(f"AUC: {auc_score}")

    print(f"Mean AUC: {np.mean(auc_scores):.5f}")
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_SMOTE_Stacking_GBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_Stacking_GBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_Stacking_GBoosting_test.csv", index = False)

## XGBoost

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_Stacking_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_Stacking_XGBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_Stacking_XGBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_Stacking_XGBoosting_test.csv", index = False)

In [ ]:
# Random Over Sampling
def KFold_ROS_Stacking_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    ros = RandomOverSampler()
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_ROS_Stacking_XGBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_Stacking_XGBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_ROS_Stacking_XGBoosting_test.csv", index = False)

In [ ]:
# Random Under Sampling
def KFold_RUS_Stacking_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    rus = RandomUnderSampler()
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_RUS_Stacking_XGBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_Stacking_XGBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_RUS_Stacking_XGBoosting_test.csv", index = False)

In [ ]:
# SMOTE
def KFold_SMOTE_Stacking_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    smote = SMOTE()
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_SMOTE_Stacking_XGBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_Stacking_XGBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_Stacking_XGBoosting_test.csv", index = False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_Stacking_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_Stacking_XGBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_Stacking_XGBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_Stacking_XGBoosting_test.csv", index = False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_Stacking_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    ros = RandomOverSampler()
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_ROS_Stacking_XGBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_Stacking_XGBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_Stacking_XGBoosting_test.csv", index = False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_Stacking_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    rus = RandomUnderSampler()
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_RUS_Stacking_XGBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_Stacking_XGBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_Stacking_XGBoosting_test.csv", index = False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_Stacking_XGBoosting_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):    
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    smote = SMOTE()
    
    clf = XGBClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth = 15,
                num_leaves=100,
                colsample_bytree=0.5,
                subsample=0.5,
                max_bin=407,
                reg_alpha=4,
                reg_lambda=5,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1
                )
    
    print("AUC:")
    oof_preds_proba = np.zeros(ftr_app.shape[0])
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("#iteration ", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]
        
        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        # Save predicted proba labels for validation set
        oof_preds_proba[valid_idx] = clf.predict_proba(valid_x)[:, 1]
        
    # return Target of train set
    df_off_preds['TARGET'] = oof_preds_proba

    # apps_all_test 
    df_test = apps_all_test.copy()
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
  
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res["TARGET"] = y_apps_test[:, 1]

    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_SMOTE_Stacking_XGBoosting_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_Stacking_XGBoosting_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_Stacking_XGBoosting_test.csv", index = False)

## Light GBM

### K-Fold Cross-Validation

In [ ]:
# No resampling method
def KFold_Stacking_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_Stacking_LightGBMC_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_Stacking_LightGBMC_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_Stacking_LightGBMC_test.csv", index = False)

In [ ]:
# Random Over Sampling
def KFold_ROS_Stacking_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    # Apply ROS to the training set
    ros = RandomOverSampler(random_state=42)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_ROS_Stacking_LightGBMC_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_ROS_Stacking_LightGBMC_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_ROS_Stacking_LightGBMC_test.csv", index = False)

In [ ]:
# Random Under Sampling
def KFold_RUS_Stacking_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    # Apply RUS to the training set
    rus = RandomUnderSampler(random_state=42)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_RUS_Stacking_LightGBMC_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_RUS_Stacking_LightGBMC_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_RUS_Stacking_LightGBMC_test.csv", index = False)

In [ ]:
# SMOTE
def KFold_SMOTE_Stacking_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = KFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    # Apply SMOTE to the training set
    smote = SMOTE()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = KFold_SMOTE_Stacking_LightGBMC_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/KFold_SMOTE_Stacking_LightGBMC_train.csv", index = False)
test_res.to_csv("/kaggle/working/KFold_SMOTE_Stacking_LightGBMC_test.csv", index = False)

### Stratified K-Fold Cross-Validation

In [ ]:
# No resampling method
def StratifiedKFold_Stacking_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_Stacking_LightGBMC_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_Stacking_LightGBMC_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_Stacking_LightGBMC_test.csv", index = False)

In [ ]:
# Random Over Sampling
def StratifiedKFold_ROS_Stacking_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    # Apply ROS to the training set
    ros = RandomOverSampler(random_state=42)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = ros.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_ROS_Stacking_LightGBMC_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_ROS_Stacking_LightGBMC_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_ROS_Stacking_LightGBMC_test.csv", index = False)

In [ ]:
# Random Under Sampling
def StratifiedKFold_RUS_Stacking_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    # Apply RUS to the training set
    rus = RandomUnderSampler(random_state=42)
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = rus.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_RUS_Stacking_LightGBMC_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_RUS_Stacking_LightGBMC_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_RUS_Stacking_LightGBMC_test.csv", index = False)

In [ ]:
# SMOTE
def StratifiedKFold_SMOTE_Stacking_LightGBMC_model(df, nfolds, random_state, drop_columns_arr, target_col, apps_all_test):
    df_off_preds = pd.DataFrame()
    df_off_preds['SK_ID_CURR'] = df['SK_ID_CURR']

    ftr_app = df.drop(drop_columns_arr, axis=1)  # feature dateset
    target_app = df[target_col]                  # target datasets
    
    folds = StratifiedKFold(n_splits = nfolds, shuffle=True, random_state = random_state)
    
    oof_preds = np.zeros(ftr_app.shape[0])
    
    clf = LGBMClassifier(
                nthread=5,
                n_estimators=50000,
                learning_rate=0.008,
                max_depth=12,
                num_leaves=58,
                colsample_bytree=0.613,
                subsample=0.8,
                max_bin=410,
                reg_alpha=3.564,
                reg_lambda=4.930,
                min_child_weight= 6,
                min_child_samples=165,
                silent=-1,
                verbose=-1,
                )
    
    # Apply SMOTE to the training set
    smote = SMOTE()
    
    for fold_idx, (train_idx , valid_idx) in enumerate(folds.split(ftr_app, target_app)):
        print("# iteration", fold_idx, 'starts')
        train_x = ftr_app.iloc[train_idx, :]
        train_y = target_app.iloc[train_idx]
        valid_x = ftr_app.iloc[valid_idx, :]
        valid_y = target_app.iloc[valid_idx]

        train_x_resampled, train_y_resampled = smote.fit_resample(train_x, train_y)

        clf.fit(train_x_resampled, train_y_resampled, eval_set=[(train_x_resampled, train_y_resampled), (valid_x, valid_y)], eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration = clf.best_iteration_)[:, 1]

    df_off_preds['TARGET'] = oof_preds

    # test set
    df_test = apps_all_test.copy() 
    test_res = pd.DataFrame()
    # save SK_ID_CURR
    test_res["SK_ID_CURR"]=df_test["SK_ID_CURR"]
    X_apps_test = df_test.drop("SK_ID_CURR", axis=1)

    y_apps_test = clf.predict_proba(X_apps_test)
    test_res[f"TARGET"] = y_apps_test[:, 1]
    return df_off_preds, clf, test_res

In [ ]:
df_off_preds, clf, test_res = StratifiedKFold_SMOTE_Stacking_LightGBMC_model(df=merged_df_train, nfolds=5, random_state=10, drop_columns_arr=['TARGET', 'SK_ID_CURR'], target_col='TARGET', apps_all_test=merged_df_test)
df_off_preds.to_csv("/kaggle/working/StratifiedKFold_SMOTE_Stacking_LightGBMC_train.csv", index = False)
test_res.to_csv("/kaggle/working/StratifiedKFold_SMOTE_Stacking_LightGBMC_test.csv", index = False)